In [1]:
import ssdeep
import pickle,os,sys,gc
import pandas as pd
# pd.set_option('display.max_columns', None)
import numpy as np
from tqdm import tqdm
from collections import Counter
import multiprocessing as mp
from multiprocessing import Process, Manager, Pool
import time,datetime
from itertools import chain
import math
from operator import itemgetter
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.metrics import jaccard_score
from sklearn.metrics.pairwise import cosine_similarity
import functools
import itertools
import operator
from itertools import chain
sys.setrecursionlimit(1000000)

In [ ]:
#  import sys,os
print(sys.executable)
print(sys.version)
print(sys.version_info)
import findspark
# os.environ['PYSPARK_PYTHON'] = '/usr/bin/python3' #'/home/antslab/anaconda3/bin/python'
findspark.init()
import pyspark
# from pyspark import SparkContext
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import Row, SQLContext, SparkSession, window
from pyspark import SparkConf, SparkContext
# Import data types
from pyspark.sql.types import *
import  pyspark.sql.functions as F
conf = SparkConf()
# conf.set("spark.local.dir", "/mnt/ssd240g/data/Leo_Spark_Home/tmp")
conf.set("spark.executor.cores","12")
conf.setMaster("local[12]")
conf.set("spark.driver.memory","85g")
conf.set("spark.executor.memory", "95g")
conf.set("spark.memory.offHeap.enabled","true")
conf.set("spark.memory.offHeap.size","85g")
conf.set("spark.driver.extraJavaOptions","-Xss80g")
conf.set("spark.driver.extraJavaOptions","-Xms80g")
# conf.set("spark.memory.storageFraction","0.2")
# conf.set("spark.driver.maxResultSize","0")
# conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")
# conf.set("spark.sql.execution.arrow.enabled", "true")
conf.set("spark.kubernetes.pyspark.pythonVersion","3")
conf.set("spark.sql.shuffle.partitions",100000)
conf.set("spark.driver.maxResultSize", "150g")

# spark = SparkSession.builder.appName("pcap_analyzer").config(conf=conf).getOrCreate() #pcap_analyzer pcapAnalyzer
spark = SparkSession.builder.master("local").appName("Test_spark").config(conf=conf).getOrCreate()
# spark.conf.set("spark.sql.execution.arrow.enabled", True)
spark.conf.set("spark.sql.debug.maxToStringFields", 290000)
spark.conf.set("spark.sql.hive.filesourcePartitionFileCacheSize",100*1024*1024*1024)

import databricks.koalas as ks
ks.options.display.max_rows = 20
ks.set_option('compute.max_rows', None)
# ks.set_option('compute.ops_on_diff_frames', True)
ks.set_option('compute.default_index_type', 'distributed')
print(ks.__version__)
spark.version

### Function

In [2]:
isp = '台灣大哥大'
time_li = ['20200106','20200107','20200108','20200109','20200110','20200111','20200112']
time = str(min(time_li))
picture_dir = '/home/antslab/NAS1_RAID6/pcap_inter/'+str(time[:4])+'_'+str(time[4:6])+'_'+str(time[6:])+'/'+isp+'/case_pictures/'


合併三個dict用update來合併以後再通過後面的df function畫圖

In [3]:

def cluster_rep_hash(proto_big_dict,proto_df_payload,proto_df,proto_loners,knee_point,date='0110',protocol='http'):
    '''
    GOAL: create each cluster's representaion ssdeep hash
    proto_df_payload: from prepare_df() function
    proto_big_dict: from similarity_compare() function
    proto_loners: loner idx list
    knee_point: from get_small_cluster() function
    date & protocol: user_defined cluster name
    
    Return: dict[cluster_name]: ssdeep hash
    '''
    upgma_dict = {}
    for key,value in proto_big_dict.items(): #cluster rep ssdeep
        target = proto_df_payload[proto_df_payload.idx == key]
        target = target.reset_index(drop=True)
        t_len = len(target)
        q2 = int(t_len*0.5)
        q2_hash = target.iloc[q2,-1]
        country = proto_df.loc[key,'country']
        ip = proto_df.loc[key,'ip_src']
        domain = proto_df.loc[key,'domain']
        if len(value)>knee_point:
            upgma_dict[str(protocol)+'_'+str(key)+'_'+str(date)+'_'+str(country)+'_'+str(domain)+'_'+str(ip)] = q2_hash
        else:
            upgma_dict[str(protocol)+'_S_'+str(key)+'_'+str(date)+'_'+str(country)+'_'+str(domain)+'_'+str(ip)] = q2_hash
    for key in proto_loners: #loner ssdeep
        target = proto_df_payload[proto_df_payload.idx == key]
        target = target.reset_index(drop=True)
        t_len = len(target)
        q2 = int(t_len*0.5)
        q2_hash = target.iloc[q2,-1]
        country = proto_df.loc[key,'country']
        ip = proto_df.loc[key,'ip_src']
        domain = proto_df.loc[key,'domain']
        upgma_dict[str(protocol)+'_L_'+str(key)+'_'+str(date)+'_'+str(country)+'_'+str(domain)+'_'+str(ip)] = q2_hash
    return upgma_dict



def pair_wise_score(upgma_dict):
    '''
    GOAL: calculate distance matrix by calculating paire-wise similarity score. 
    and pick upper triangle convert to list
    Input: from cluster_rep_hash() function
    ['cluster']: name
    ['ssdeep']: cluster's representation hash
    
    Return: df=>['c_ssdeep_li']:the hashes list compare to, ['score']:list of distances (upper-triangle, exclude self)
    '''
    def compare(target_hash,candidate_hash_li):
        score_li = []
        for c_hash in candidate_hash_li:
            score_li.append(100-ssdeep.compare(target_hash,c_hash)) #相似度滿分100，轉換成距離最近0
        return score_li
    used_idx = []
    def create_hash_li(t_hash):
        idx_set = set(upgma_df[upgma_df.ssdeep == t_hash].index)#[0]
        same_hash_li = sorted(list(idx_set - set(used_idx)))
        idx = same_hash_li[0]
        used_idx.append(idx)
        return upgma_df.loc[idx+1:]['ssdeep'].tolist()
    upgma_df = pd.DataFrame(upgma_dict.items(),columns=['cluster','ssdeep'])
    upgma_df['c_ssdeep_li'] = upgma_df.ssdeep.map(create_hash_li)
    upgma_df['score'] = upgma_df.apply(lambda x: compare(x.ssdeep, x.c_ssdeep_li), axis=1)
    return upgma_df



def draw_upgma(upgma_df,picture_dir=picture_dir,name='upgma'):
    '''
    GOAL: using upper triangle's distance to draw upgma
    Input: from pair_wise_score() function
    Output: diagram of UPGMA、Z info
    '''
    if not os.path.exists(picture_dir):
        os.makedirs(picture_dir,exist_ok=True) 
    score_li = upgma_df['score'].tolist()
    score_li = list(filter(None, score_li))
    score_li = sum(score_li,[])
    Z = linkage(score_li, 'average')
    fig = plt.figure(figsize=(60, 24)) #(25,10) #(5,2)
    # plt.savefig(fig)
    dn = dendrogram(Z,labels=upgma_df.cluster.tolist())
    plt.savefig(picture_dir+str(name)+'.png', dpi=600, format='png', bbox_inches='tight')
    return dn,Z

In [ ]:
# 大家的pickle_dir都不同0109 0110 0111
# not save?

# time_li = ['02','10','30']
protocols_need = ['http','mysql','ftp','smb','smtp','imap','pop','rpc','ssh','telnet','sip']
# proto = HTTP, FTP, IMAP, MySQL, POP, RPC, SIP, SMB, SMTP, SSH, Telnet
for proto in protocols_need:
    three_dict = {}
    for time in time_li:
        path = '/home/antslab/NAS1_RAID6/pcap_inter/'+str(time[:4])+'_'+str(time[4:6])+'_'+str(time[6:])+'/'+isp+'/case_pickles/'+proto+'_upgma_dict_'+str(time)+'.pkl'
        try:
            temp = pickle.load(open(path,'rb'))
            three_dict.update(temp)
        except FileNotFoundError:
            print("!!! No file:",path,'!!!')
            continue


    loner_dict = {}
    for k,v in three_dict.items():
        if '_L_' in k:
            loner_dict[k]=v
        if '_S_' in k:
            loner_dict[k] =v
    time_li2 = [int(x) for x in time_li]
    min_time  = str(min(time_li2))
    max_time = str(max(time_li2))
    filename = "_".join(time_li)
    pickle.dump(obj=(three_dict,loner_dict),file=open('/home/antslab/NAS1_RAID6/pcap_inter/'+str(min_time[:4])+'_'+str(min_time[4:6])+'_'+str(min_time[6:])+'/'+isp+'/case_pickles/combine_dict_loner_dict_'+filename+'.pkl','wb'))
    three_upgma_df = pair_wise_score(three_dict) #改!!
    dn,Z = draw_upgma(three_upgma_df,
               picture_dir='/home/antslab/NAS1_RAID6/pcap_inter/'+str(min_time[:4])+'_'+str(min_time[4:6])+'_'+str(min_time[6:])+'/'+isp+'/case_pictures/' #改!!
               ,name=str(min_time)+'-'+str(max_time)+'_upgma') #改!! 0106-0112_upgma            
loner_dict

!!! No file: /home/antslab/NAS1_RAID6/pcap_inter/2020_01_08/台灣大哥大/case_pickles/pop_upgma_dict_20200108.pkl !!!
!!! No file: /home/antslab/NAS1_RAID6/pcap_inter/2020_01_11/台灣大哥大/case_pickles/ssh_upgma_dict_20200111.pkl !!!


In [ ]:
# three_upgma_df = pair_wise_score(three_dict) #改!!
# dn,Z = draw_upgma(three_upgma_df,
#            picture_dir='/home/antslab/spark_data/pcap_inter/2020_01_06/中華電信/case_pictures/' #改!!
#            ,name='0106-0112_upgma') #改!! 0106-0112_upgma

In [ ]:
Z

## 畫跨天protocol大小圖

In [3]:
trend_dict_time_all = {}
trend_dict_country_all = {}
trend_dict_ip_all = {}
trend_dict_ssdeep_all = {}
def draw_trend_pic(date,proto,pickle_dir,trend_dict_time_all=trend_dict_time_all,trend_dict_country_all=trend_dict_country_all,trend_dict_ip_all=trend_dict_ip_all,trend_dict_ssdeep_all=trend_dict_ssdeep_all):
    '''
    GOAL: output dict to draw trend-circle picture
    
    Return: 4 dict for the same proto for the next day
    '''
    try:
        (proto_df, proto_df_payload,proto_big_dict,proto_loners,proto_score,proto_cluster_score_dict,
         proto_upgma_dict,stat_df) = pickle.load(open(pickle_dir+str(date)+'_'+str(proto)+'_all.pkl','rb'))
    except ValueError:
        try:
            (proto_df, proto_df_payload,proto_big_dict,proto_loners,proto_score,proto_cluster_score_dict,
             proto_upgma_dict,stat_df,df2) = pickle.load(open(pickle_dir+str(date)+'_'+str(proto)+'_all.pkl','rb'))
        except ValueError:
            try:
                (proto_df, proto_df_payload,proto_big_dict,proto_loners,proto_score,proto_cluster_score_dict,
                                        proto_upgma_dict,stat_df) = pickle.load(open(pickle_dir+str(date)+'_'+str(proto)+'_all.pkl','rb'))
            except ValueError:
                (proto_df, proto_df_payload,proto_big_dict,proto_loners,proto_score,proto_cluster_score_dict,
                                        proto_upgma_dict) = pickle.load(open(pickle_dir+str(date)+'_'+str(proto)+'_all.pkl','rb'))
    except FileNotFoundError:
        print("!!File Not Found:",date,proto,"!!")
        return trend_dict_time_all,trend_dict_country_all,trend_dict_ip_all,trend_dict_ssdeep_all


    def ssdeep_compare(target_hash,candidate_df):
        '''
        Input1: string of hash
        Input2: dataframe of candidate
        '''
        def compare(candidate_hash):
            if type(candidate_hash) == str:
                return ssdeep.compare(target_hash,candidate_hash)
            else:
                score_li = []
                for c_h in candidate_hash:
                    score_li.append(ssdeep.compare(target_hash,c_h))
                return max(score_li)
        return candidate_df.hash.map(compare)

    trend_dict_time = {}
    trend_dict_country = {}
    trend_dict_ip = {}
    trend_dict_ssdeep = {}
    candidate_df = pd.DataFrame(trend_dict_ssdeep_all.items(),columns=['idx','hash']) #其他已經有的cluster
    for key,value in proto_big_dict.items():
        target = proto_df_payload[proto_df_payload.idx == key]
        target = target.reset_index(drop=True)
        t_len = len(target)
        q2 = int(t_len*0.5)
        t_q2_key = target.iloc[q2,-1] #要跟別人比較的cluster hash
        if trend_dict_ssdeep_all == {}: #第一次
            member_li = value[:]
            
            t_q2_member1 = proto_df_payload[proto_df_payload.idx == member_li[-1]] # member list最後一個
            t_q2_member1 = t_q2_member1.reset_index(drop=True)
            t_len = len(t_q2_member1)
            q2 = int(t_len*0.5)
            t_q2_member1 = t_q2_member1.iloc[q2,-1]
            
            t_q2_member2 = proto_df_payload[proto_df_payload.idx == max(member_li)] #member list最大的那個
            t_q2_member2 = t_q2_member2.reset_index(drop=True)
            t_len = len(t_q2_member2)
            q2 = int(t_len*0.5)
            t_q2_member2 = t_q2_member2.iloc[q2,-1]
            
            trend_dict_ssdeep[str(date)+"_"+str(key)] = [t_q2_key,t_q2_member1,t_q2_member2] #可以增加hash candidate?
            member_li.append(key)
            select_df = proto_df.loc[member_li]
            time_li = select_df.session_time.tolist()
            trend_dict_time[str(date)+"_"+str(key)] = time_li
            country_li = select_df.country.tolist()
            ip_li = select_df.ip_src.tolist()
            assert len(time_li) == len(country_li) == len(ip_li)
            trend_dict_country[str(date)+"_"+str(key)] = country_li
            trend_dict_ip[str(date)+"_"+str(key)] = ip_li
        else: #後面幾次
            candidate_df['score'] = ssdeep_compare(t_q2_key,candidate_df) 
            max_score = candidate_df.score.max()
            if max_score < 1: #都沒有相近的
                
                member_li = value[:]
                
                t_q2_member1 = proto_df_payload[proto_df_payload.idx == member_li[-1]] # member list最後一個
                t_q2_member1 = t_q2_member1.reset_index(drop=True)
                t_len = len(t_q2_member1)
                q2 = int(t_len*0.5)
                t_q2_member1 = t_q2_member1.iloc[q2,-1]

                t_q2_member2 = proto_df_payload[proto_df_payload.idx == max(member_li)] #member list最大的那個
                t_q2_member2 = t_q2_member2.reset_index(drop=True)
                t_len = len(t_q2_member2)
                q2 = int(t_len*0.5)
                t_q2_member2 = t_q2_member2.iloc[q2,-1]
                
                trend_dict_ssdeep[str(date)+"_"+str(key)] = [t_q2_key,t_q2_member1,t_q2_member2]
                member_li.append(key)
                select_df = proto_df.loc[member_li]
                time_li = select_df.session_time.tolist()
                trend_dict_time[str(date)+"_"+str(key)] = time_li
                country_li = select_df.country.tolist()
                ip_li = select_df.ip_src.tolist()
                assert len(time_li) == len(country_li) == len(ip_li)
                trend_dict_country[str(date)+"_"+str(key)] = country_li
                trend_dict_ip[str(date)+"_"+str(key)] = ip_li
            elif max_score>0: #有相近的合併到原本的dict
                try:
                    combine_id = candidate_df[candidate_df.score == max_score].idx.tolist()[0]
                except:
                    print( candidate_df,max_score,target)
                ori_ssdeep_list = trend_dict_ssdeep_all[combine_id][:]
                if type(ori_ssdeep_list) == str:
                    ori_ssdeep_list = [ori_ssdeep_list] #之前只有一個ssdeep hash
                ori_time_list = trend_dict_time_all[combine_id][:]
                ori_country_list = trend_dict_country_all[combine_id][:]
                ori_ip_list = trend_dict_ip_all[combine_id][:]
                member_li = value[:]
                
                t_q2_member1 = proto_df_payload[proto_df_payload.idx == member_li[-1]] # member list最後一個
                t_q2_member1 = t_q2_member1.reset_index(drop=True)
                t_len = len(t_q2_member1)
                q2 = int(t_len*0.5)
                t_q2_member1 = t_q2_member1.iloc[q2,-1]

                t_q2_member2 = proto_df_payload[proto_df_payload.idx == max(member_li)] #member list最大的那個
                t_q2_member2 = t_q2_member2.reset_index(drop=True)
                t_len = len(t_q2_member2)
                q2 = int(t_len*0.5)
                t_q2_member2 = t_q2_member2.iloc[q2,-1]                
                
                t_q2 = [t_q2_key,t_q2_member1,t_q2_member2]
                member_li.append(key)
                select_df = proto_df.loc[member_li]
                time_li = select_df.session_time.tolist()
                ori_time_list.extend(time_li)
                country_li = select_df.country.tolist()
                ip_li = select_df.ip_src.tolist()
                ori_country_list.extend(country_li)
                ori_ip_list.extend(ip_li)
                ori_ssdeep_list.extend(t_q2) #我合併進入別人的群，所以把我群的key hash也加入
                assert len(ori_time_list) == len(ori_country_list) == len(ori_ip_list)
                trend_dict_time_all[combine_id] = ori_time_list
                trend_dict_country_all[combine_id] = ori_country_list
                trend_dict_ip_all[combine_id] = ori_ip_list
                trend_dict_ssdeep_all[combine_id] = ori_ssdeep_list
            else:
                print(max_score)
#         break #debug
    trend_dict_time_all.update(trend_dict_time)
    trend_dict_country_all.update(trend_dict_country)
    trend_dict_ip_all.update(trend_dict_ip)
    trend_dict_ssdeep_all.update(trend_dict_ssdeep)
    return trend_dict_time_all,trend_dict_country_all,trend_dict_ip_all,trend_dict_ssdeep_all

In [4]:
def sort_li(time_li, country_li):
    '''
    GOAL: sort by time (align with time's order)
    Return: list
    '''
    sort_country_li = [x for _,x in sorted(zip(time_li,country_li))]
    return sort_country_li

In [5]:

# 指定跨天日期們
date_li = ["20200106","20200107","20200108","20200109","20200110","20200111","20200112"] #,"0102","0130"
# date_li = ["0102","0106","0107","0108","0109","0110","0111","0112","0130"] #,"0102","0130"
# date_li = ['0102','0130','0110']
# isp = '亞太電信'
proto_li =  ['http','mysql','ftp','smb','smtp','imap','pop','rpc','ssh','telnet','sip']
# proto_li = ['tds']
for proto in tqdm(proto_li):
    trend_dict_time_all = {}
    trend_dict_country_all = {}
    trend_dict_ip_all = {}
    trend_dict_ssdeep_all = {}
    for date in date_li:
        pickle_dir = '/home/antslab/NAS1_RAID6/pcap_inter/'+str(date[:4])+'_'+str(date[4:6])+'_'+str(date[6:])+'/'+isp+'/case_pickles/'
        trend_dict_time_all,trend_dict_country_all,trend_dict_ip_all,trend_dict_ssdeep_all = draw_trend_pic(date,proto,
                                                                                          pickle_dir,trend_dict_time_all,trend_dict_country_all,trend_dict_ip_all,trend_dict_ssdeep_all)
    trend_dict_time_all = {k: v for k, v in sorted(trend_dict_time_all.items(), key=lambda item: len(item[1]),reverse=True)}
    time_df = pd.DataFrame(trend_dict_time_all.items(),columns=['idx','timestamp'])
    trend_dict_country_all = {k: v for k, v in sorted(trend_dict_country_all.items(), key=lambda item: len(item[1]),reverse=True)}
    country_df = pd.DataFrame(trend_dict_country_all.items(),columns=['idx','country'])
    trend_dict_ip_all = {k: v for k, v in sorted(trend_dict_ip_all.items(), key=lambda item: len(item[1]),reverse=True)}
    ip_df =  pd.DataFrame(trend_dict_ip_all.items(),columns=['idx','src_ip'])
    ssdeep_df = pd.DataFrame(trend_dict_ssdeep_all.items(),columns=['idx','ssdeep'])
    all_df = pd.merge(time_df,country_df,on='idx')
    all_df =  pd.merge(all_df,ip_df,on='idx')
    all_df = pd.merge(all_df,ssdeep_df,on='idx')
    all_df['country'] = all_df.apply(lambda x: sort_li(x.timestamp, x.country), axis=1)
    all_df['src_ip'] = all_df.apply(lambda x: sort_li(x.timestamp, x.src_ip), axis=1)
    all_df['timestamp'] = all_df.timestamp.map(sorted)
    file_name = "_".join(sorted(date_li))
    date_li2 = [int(x) for x in date_li]
    min_date = str(min(date_li2))
    pickle.dump(obj=all_df,
                file=open('/home/antslab/NAS1_RAID6/pcap_inter/'+str(min_date[:4])+'_'+str(min_date[4:6])+'_'+str(min_date[6:])+'/'+isp+'/case_pickles/'+proto+'_trend_df_'+file_name+'.pkl','wb'))
    print("Protocol Pattern draw save path:",'/home/antslab/NAS1_RAID6/pcap_inter/'+str(min_date[:4])+'_'+str(min_date[4:6])+'_'+str(min_date[6:])+'/'+isp+'/case_pickles/'+proto+'_trend_df_'+file_name+'.pkl')

 18%|█▊        | 2/11 [02:12<13:52, 92.52s/it] 

Protocol Pattern draw save path: /home/antslab/NAS1_RAID6/pcap_inter/2020_01_06/台灣大哥大/case_pickles/http_trend_df_20200106_20200107_20200108_20200109_20200110_20200111_20200112.pkl
Protocol Pattern draw save path: /home/antslab/NAS1_RAID6/pcap_inter/2020_01_06/台灣大哥大/case_pickles/mysql_trend_df_20200106_20200107_20200108_20200109_20200110_20200111_20200112.pkl


 27%|██▋       | 3/11 [02:13<08:40, 65.08s/it]

Protocol Pattern draw save path: /home/antslab/NAS1_RAID6/pcap_inter/2020_01_06/台灣大哥大/case_pickles/ftp_trend_df_20200106_20200107_20200108_20200109_20200110_20200111_20200112.pkl


 36%|███▋      | 4/11 [02:27<05:49, 49.89s/it]

...

Protocol Pattern draw save path: /home/antslab/NAS1_RAID6/pcap_inter/2020_01_06/台灣大哥大/case_pickles/ssh_trend_df_20200106_20200107_20200108_20200109_20200110_20200111_20200112.pkl
!!File Not Found: 20200108 telnet !!


 91%|█████████ | 10/11 [2:02:55<25:23, 1523.49s/it] 

Protocol Pattern draw save path: /home/antslab/NAS1_RAID6/pcap_inter/2020_01_06/台灣大哥大/case_pickles/telnet_trend_df_20200106_20200107_20200108_20200109_20200110_20200111_20200112.pkl
!!File Not Found: 20200108 sip !!


100%|██████████| 11/11 [2:02:56<00:00, 670.58s/it] 

Protocol Pattern draw save path: /home/antslab/NAS1_RAID6/pcap_inter/2020_01_06/台灣大哥大/case_pickles/sip_trend_df_20200106_20200107_20200108_20200109_20200110_20200111_20200112.pkl


In [ ]:
all_df

In [ ]:
# trend_dict_time_all = {}
# trend_dict_country_all = {}
# trend_dict_ip_all = {}
# trend_dict_ssdeep_all = {}
# # def draw_trend_pic(date,proto,pickle_dir,trend_dict_time_all=trend_dict_time_all,trend_dict_country_all=trend_dict_country_all,trend_dict_ip_all=trend_dict_ip_all,trend_dict_ssdeep_all=trend_dict_ssdeep_all):
# #     '''
# #     GOAL: output dict to draw trend-circle picture
    
# #     Return: 4 dict for the same proto for the next day
# #     '''
# date = '20200106'
# proto = 'http'
# pickle_dir = '/home/antslab/NAS1_RAID6/pcap_inter/'+str(date[:4])+'_'+str(date[4:6])+'_'+str(date[6:])+'/'+isp+'/case_pickles/'
# (proto_df, proto_df_payload,proto_big_dict,proto_loners,proto_score,proto_cluster_score_dict,
#  proto_upgma_dict,stat_df,df2) = pickle.load(open(pickle_dir+str(date)+'_'+str(proto)+'_all.pkl','rb'))



# def ssdeep_compare(target_hash,candidate_df):
#     '''
#     Input1: string of hash
#     Input2: dataframe of candidate
#     '''
#     def compare(candidate_hash):
#         if type(candidate_hash) == str:
#             return ssdeep.compare(target_hash,candidate_hash)
#         else:
#             score_li = []
#             for c_h in candidate_hash:
#                 score_li.append(ssdeep.compare(target_hash,c_h))
#             return max(score_li)
#     return candidate_df.hash.map(compare)

# trend_dict_time = {}
# trend_dict_country = {}
# trend_dict_ip = {}
# trend_dict_ssdeep = {}
# candidate_df = pd.DataFrame(trend_dict_ssdeep_all.items(),columns=['idx','hash']) #其他已經有的cluster
# for key,value in proto_big_dict.items():
#     target = proto_df_payload[proto_df_payload.idx == key]
#     target = target.reset_index(drop=True)
#     t_len = len(target)
#     q2 = int(t_len*0.5)
#     t_q2_key = target.iloc[q2,-1] #要跟別人比較的cluster hash
#     if trend_dict_ssdeep_all == {}: #第一次
#         member_li = value[:]

#         t_q2_member1 = proto_df_payload[proto_df_payload.idx == member_li[-1]] # member list最後一個
#         t_q2_member1 = t_q2_member1.reset_index(drop=True)
#         t_len = len(t_q2_member1)
#         q2 = int(t_len*0.5)
#         t_q2_member1 = t_q2_member1.iloc[q2,-1]

#         t_q2_member2 = proto_df_payload[proto_df_payload.idx == max(member_li)] #member list最大的那個
#         t_q2_member2 = t_q2_member2.reset_index(drop=True)
#         t_len = len(t_q2_member2)
#         q2 = int(t_len*0.5)
#         t_q2_member2 = t_q2_member2.iloc[q2,-1]

#         trend_dict_ssdeep[str(date)+"_"+str(key)] = [t_q2_key,t_q2_member1,t_q2_member2] #可以增加hash candidate?
#         member_li.append(key)
#         select_df = proto_df.loc[member_li]
#         time_li = select_df.session_time.tolist()
#         trend_dict_time[str(date)+"_"+str(key)] = time_li
#         country_li = select_df.country.tolist()
#         ip_li = select_df.ip_src.tolist()
#         assert len(time_li) == len(country_li) == len(ip_li)
#         trend_dict_country[str(date)+"_"+str(key)] = country_li
#         trend_dict_ip[str(date)+"_"+str(key)] = ip_li
#     else: #後面幾次
#         candidate_df['score'] = ssdeep_compare(t_q2,candidate_df) 
#         max_score = candidate_df.score.max()
#         if max_score < 1: #都沒有相近的

#             member_li = value[:]

#             t_q2_member1 = proto_df_payload[proto_df_payload.idx == member_li[-1]] # member list最後一個
#             t_q2_member1 = t_q2_member1.reset_index(drop=True)
#             t_len = len(t_q2_member1)
#             q2 = int(t_len*0.5)
#             t_q2_member1 = t_q2_member1.iloc[q2,-1]

#             t_q2_member2 = proto_df_payload[proto_df_payload.idx == max(member_li)] #member list最大的那個
#             t_q2_member2 = t_q2_member2.reset_index(drop=True)
#             t_len = len(t_q2_member2)
#             q2 = int(t_len*0.5)
#             t_q2_member2 = t_q2_member2.iloc[q2,-1]

#             trend_dict_ssdeep[str(date)+"_"+str(key)] = [t_q2_key,t_q2_member1,t_q2_member2]
#             member_li.append(key)
#             select_df = proto_df.loc[member_li]
#             time_li = select_df.session_time.tolist()
#             trend_dict_time[str(date)+"_"+str(key)] = time_li
#             country_li = select_df.country.tolist()
#             ip_li = select_df.ip_src.tolist()
#             assert len(time_li) == len(country_li) == len(ip_li)
#             trend_dict_country[str(date)+"_"+str(key)] = country_li
#             trend_dict_ip[str(date)+"_"+str(key)] = ip_li
#         elif max_score>0: #有相近的合併到原本的dict
#             try:
#                 combine_id = candidate_df[candidate_df.score == max_score].idx.tolist()[0]
#             except:
#                 print( candidate_df,max_score,target)
#             ori_ssdeep_list = trend_dict_ssdeep_all[combine_id][:]
#             if type(ori_ssdeep_list) == str:
#                 ori_ssdeep_list = [ori_ssdeep_list] #之前只有一個ssdeep hash
#             ori_time_list = trend_dict_time_all[combine_id][:]
#             ori_country_list = trend_dict_country_all[combine_id][:]
#             ori_ip_list = trend_dict_ip_all[combine_id][:]
#             member_li = value[:]

#             t_q2_member1 = proto_df_payload[proto_df_payload.idx == member_li[-1]] # member list最後一個
#             t_q2_member1 = t_q2_member1.reset_index(drop=True)
#             t_len = len(t_q2_member1)
#             q2 = int(t_len*0.5)
#             t_q2_member1 = t_q2_member1.iloc[q2,-1]

#             t_q2_member2 = proto_df_payload[proto_df_payload.idx == max(member_li)] #member list最大的那個
#             t_q2_member2 = t_q2_member2.reset_index(drop=True)
#             t_len = len(t_q2_member2)
#             q2 = int(t_len*0.5)
#             t_q2_member2 = t_q2_member2.iloc[q2,-1]                

#             t_q2 = [t_q2_key,t_q2_member1,t_q2_member2]
#             member_li.append(key)
#             select_df = proto_df.loc[member_li]
#             time_li = select_df.session_time.tolist()
#             ori_time_list.extend(time_li)
#             country_li = select_df.country.tolist()
#             ip_li = select_df.ip_src.tolist()
#             ori_country_list.extend(country_li)
#             ori_ip_list.extend(ip_li)
#             ori_ssdeep_list.extend(t_q2) #我合併進入別人的群，所以把我群的key hash也加入
#             assert len(ori_time_list) == len(ori_country_list) == len(ori_ip_list)
#             trend_dict_time_all[combine_id] = ori_time_list
#             trend_dict_country_all[combine_id] = ori_country_list
#             trend_dict_ip_all[combine_id] = ori_ip_list
#             trend_dict_ssdeep_all[combine_id] = ori_ssdeep_list
#         else:
#             print(max_score)
# #         break #debug
# trend_dict_time_all.update(trend_dict_time)
# trend_dict_country_all.update(trend_dict_country)
# trend_dict_ip_all.update(trend_dict_ip)
# trend_dict_ssdeep_all.update(trend_dict_ssdeep)
# # return trend_dict_time_all,trend_dict_country_all,trend_dict_ip_all,trend_dict_ssdeep_all

輸出cluster之key session的time list

In [6]:
date_li = ["20200106","20200107","20200108","20200109","20200110","20200111","20200112"]
min(date_li)

'20200106'

In [ ]:
proto_li = ['http','mysql','ftp','smb','smtp','imap','pop','rpc','ssh','telnet','sip']
#'http','smb','telnet','ftp','smtp','mysql','ssh','rpc','imap','pop','sip'
# proto = 'tds' 'tds','http',

# date_li = ["0102","0106","0107","0108","0109","0110","0111","0112","0130"] #,"0102","0130"
date_li = ["20200106","20200107","20200108","20200109","20200110","20200111","20200112"] #,"0102","0130"
# date_li = ['0102','0130','0110']
file_name = "_".join(sorted(date_li))
min_date = str(min(date_li))
for proto in tqdm(proto_li):
    http_df = pickle.load(open('/home/antslab/NAS1_RAID6/pcap_inter/'+str(min_date[:4])+'_'+str(min_date[4:6])+'_'+str(min_date[6:])+'/'+isp+'/case_pickles/'+proto+'_trend_df_'+file_name+'.pkl','rb'))

    #輸出cluster之key session的time list
    wireshark_li = http_df.idx.tolist() #.head(15)前15大cluster #改!proto
    wireshark_rank = []
    for i,v in enumerate(wireshark_li):
        wireshark_rank.append(i+1)
    wireshark_rank = [x for _,x in sorted(zip(wireshark_li,wireshark_rank))]
    wireshark_li = sorted(wireshark_li)

    save_path_li = []
    now_date = '00' #現在正在處理的日期
    for i,wireshark in zip(wireshark_rank,wireshark_li):
        date = wireshark.split('_')[0] #該cluster key的同月份日期
        if date!= now_date: #新日期才要重讀
            now_date = date

            pickle_dir = '/home/antslab/NAS1_RAID6/pcap_inter/'+str(date[:4])+'_'+str(date[4:6])+'_'+str(date[6:])+'/'+isp+'/case_pickles/'
            try:
                (proto_df, proto_df_payload,proto_big_dict,proto_loners,proto_score,proto_cluster_score_dict,
                 proto_upgma_dict,stat_df,df2) = pickle.load(open(pickle_dir+str(date)+'_'+str(proto)+'_all.pkl','rb'))
            except ValueError:
                try:
                    (proto_df, proto_df_payload,proto_big_dict,proto_loners,proto_score,proto_cluster_score_dict,
                                            proto_upgma_dict,stat_df) = pickle.load(open(pickle_dir+str(date)+'_'+str(proto)+'_all.pkl','rb'))
                except ValueError:
                    (proto_df, proto_df_payload,proto_big_dict,proto_loners,proto_score,proto_cluster_score_dict,
                                            proto_upgma_dict) = pickle.load(open(pickle_dir+str(date)+'_'+str(proto)+'_all.pkl','rb'))
            except FileNotFoundError:
                print("!!File Not Found:",date,proto,"!!")
        idx = wireshark.split('_')[-1] #該cluster在該日期該proto的df中的index
        #     else:
        try:
            time_list = proto_df.loc[int(idx),'session_time_list'].tolist()
        except AttributeError:
            time_list = proto_df.loc[int(idx),'session_time_list']
        timelist_path = pickle_dir+'timelist_'+proto+'_large#'+str(i)+'_clusterID#'+str(idx)+'.pkl'
        pickle.dump(file=open(timelist_path,'wb'),obj=time_list)
#         print(wireshark,'save in:',timelist_path) #truly save path
        save_path_li.append(timelist_path)
    try:
        first_date = wireshark_li[0].split('_')[0]#[-2:]
        save_path = '/home/antslab/NAS1_RAID6/pcap_inter/'+str(first_date[:4])+'_'+str(first_date[4:6])+'_'+str(first_date[6:])+'/'+isp+'/case_pickles/'+proto+'_clusterKey_timelist_paths_'+file_name+'.pkl'
        pickle.dump(file=open(save_path,'wb'),obj=save_path_li)  
        print(proto,':',save_path) #for 証鴻 pickle save path
    except IndexError:
        print(proto,"Not Save, because it's empty.")


# http_df

  9%|▉         | 1/11 [03:44<37:24, 224.42s/it]

http : /home/antslab/NAS1_RAID6/pcap_inter/2020_01_06/台灣大哥大/case_pickles/http_clusterKey_timelist_paths_20200106_20200107_20200108_20200109_20200110_20200111_20200112.pkl


 18%|█▊        | 2/11 [03:46<23:39, 157.73s/it]

mysql : /home/antslab/NAS1_RAID6/pcap_inter/2020_01_06/台灣大哥大/case_pickles/mysql_clusterKey_timelist_paths_20200106_20200107_20200108_20200109_20200110_20200111_20200112.pkl


 27%|██▋       | 3/11 [03:47<14:44, 110.60s/it]

...

smb : /home/antslab/NAS1_RAID6/pcap_inter/2020_01_06/台灣大哥大/case_pickles/smb_clusterKey_timelist_paths_20200106_20200107_20200108_20200109_20200110_20200111_20200112.pkl


 45%|████▌     | 5/11 [03:55<05:36, 56.06s/it]

smtp : /home/antslab/NAS1_RAID6/pcap_inter/2020_01_06/台灣大哥大/case_pickles/smtp_clusterKey_timelist_paths_20200106_20200107_20200108_20200109_20200110_20200111_20200112.pkl
imap : /home/antslab/NAS1_RAID6/pcap_inter/2020_01_06/台灣大哥大/case_pickles/imap_clusterKey_timelist_paths_20200106_20200107_20200108_20200109_20200110_20200111_20200112.pkl


 73%|███████▎  | 8/11 [03:55<01:22, 27.55s/it]

pop : /home/antslab/NAS1_RAID6/pcap_inter/2020_01_06/台灣大哥大/case_pickles/pop_clusterKey_timelist_paths_20200106_20200107_20200108_20200109_20200110_20200111_20200112.pkl
rpc : /home/antslab/NAS1_RAID6/pcap_inter/2020_01_06/台灣大哥大/case_pickles/rpc_clusterKey_timelist_paths_20200106_20200107_20200108_20200109_20200110_20200111_20200112.pkl


In [61]:
save_path_li

[]

In [ ]:
# def timestamp_cut_country(time_li,country_li):
#     thr_lower = 1577887200
#     thr_upper = 1577984400 #2020-01-03 01
#     c_li = []
# #     i_li = []
#     for t,c,i in zip(time_li,country_li):
#         if thr_lower<t<thr_upper:
#             c_li.append(c)
# #             i_li.append(i)
#     return c_li

# def timestamp_cut_ip(time_li,country_li,ip_li):
#     thr_lower = 1577887200
#     thr_upper = 1577984400 #2020-01-03 01
#     c_li = []
#     i_li = []
#     for t,c,i in zip(time_li,country_li,ip_li):
#         if 1577887200<t<thr_upper:
#             c_li.append(c)
#             i_li.append(i)
#     return i_li

In [ ]:
# time.mktime(time.strptime('2020-01-03 01','%Y-%m-%d %H'))

In [ ]:
# #輸出cluster之key session的time list
# wireshark_li = http_df.idx.tolist() #.head(15)前15大cluster #改!proto
# wireshark_rank = []
# for i,v in enumerate(wireshark_li):
#     wireshark_rank.append(i+1)
# wireshark_rank = [x for _,x in sorted(zip(wireshark_li,wireshark_rank))]
# wireshark_li = sorted(wireshark_li)

# save_path_li = []
# now_date = '00' #現在正在處理的日期
# for i,wireshark in tqdm(zip(wireshark_rank,wireshark_li)):
#     date = wireshark.split('_')[0] #該cluster key的同月份日期
#     if date!= now_date: #新日期才要重讀
#         now_date = date
        
#         pickle_dir = '/home/antslab/spark_data/pcap_inter/2020_01_'+str(date[-2:])+'/中華電信/case_pickles/'
#         try:
#             (proto_df, proto_df_payload,proto_big_dict,proto_loners,proto_score,proto_cluster_score_dict,
#                                     proto_upgma_dict,stat_df) = pickle.load(open(pickle_dir+str(date)+'_'+str(proto)+'_all.pkl','rb'))
#         except ValueError:
#             (proto_df, proto_df_payload,proto_big_dict,proto_loners,proto_score,proto_cluster_score_dict,
#                                     proto_upgma_dict) = pickle.load(open(pickle_dir+str(date)+'_'+str(proto)+'_all.pkl','rb'))
#         except FileNotFoundError:
#             continue
#     idx = wireshark.split('_')[-1] #該cluster在該日期該proto的df中的index
#     #     else:
#     try:
#         time_list = proto_df.loc[int(idx),'session_time_list'].tolist()
#     except AttributeError:
#         time_list = proto_df.loc[int(idx),'session_time_list']
#     timelist_path = pickle_dir+'timelist_'+proto+'_large#'+str(i)+'_clusterID#'+str(idx)+'.pkl'
#     pickle.dump(file=open(timelist_path,'wb'),obj=time_list)
#     print(wireshark,'save in:',timelist_path)
#     save_path_li.append(timelist_path)

# first_date = wireshark_li[0].split('_')[0][-2:]
# save_path = '/home/antslab/spark_data/pcap_inter/2020_01_'+first_date+'/中華電信/case_pickles/'+proto+'_clusterKey_timelist_paths_'+file_name+'.pkl'
# pickle.dump(file=open(save_path,'wb'),obj=save_path_li)  
# print('all paths in:',save_path)

#### 新關聯方法AS
* topology
    * 我們會先找所給定期間的指定所有protocols之所有sessions與對應IPs
    * 接下來會利用此段期間的各IP，去尋找這個IP在這段期間做的手法(攻擊樣態群集)
    * 找出不同IP所橫跨對應的攻擊樣態群，計算jaccard相似度
    * 將所採用相似手法(score>thr)的IP群聚

In [ ]:
#第一次只要跑這格。
proto_li =  ['http','mysql','ftp','smb','smtp','imap','pop','rpc','ssh','telnet','sip'] #改!!'tds',
# date_li2 = ["0106","0107","0108","0109","0110","0111","0112"] 
date_li = ["20200106","20200107","20200108","20200109","20200110","20200111","20200112"]  #改!!

file_name = "_".join(sorted(date_li)) #改!!
min_date = str(min(date_li))
max_date = str(max(date_li))
for i,proto in enumerate(proto_li):
    if i==0:
        all_df = pickle.load(file=open('/home/antslab/NAS1_RAID6/pcap_inter/'+str(min_date[:4])+'_'+str(min_date[4:6])+'_'+str(min_date[6:])+'/'+isp+'/case_pickles/'+proto+'_trend_df_'+file_name+'.pkl','rb')) #改!!)
        all_df['idx'] = all_df['idx']+'_'+proto
    else:
        temp = pickle.load(file=open('/home/antslab/NAS1_RAID6/pcap_inter/'+str(min_date[:4])+'_'+str(min_date[4:6])+'_'+str(min_date[6:])+'/'+isp+'/case_pickles/'+proto+'_trend_df_'+file_name+'.pkl','rb')) #改!!
        temp['idx'] = temp['idx']+'_'+proto
        all_df = all_df.append(temp)
all_df = all_df.reset_index(drop=True)
#首次須先輸出noise pkl給証鴻，另外處理後才會獲得noise_clusters
pickle_dir = '/home/antslab/NAS1_RAID6/pcap_inter/'+str(min_date[:4])+'_'+str(min_date[4:6])+'_'+str(min_date[6:])+'/'+isp+'/case_pickles/'
pickle.dump(file=open(pickle_dir+'clusterName_overview_denoise_df_'+str(min_date)+'_'+str(date_li[-1])+'.pkl','wb'),obj=all_df)
print("Denoise path save in:",
      pickle_dir+'clusterName_overview_denoise_df_'+str(min_date)+'_'+str(date_li[-1])+'.pkl')
all_df

In [9]:
print("Denoise path save in:",
      pickle_dir+'clusterName_overview_denoise_df_'+str(min_date)+'_'+str(date_li[-1])+'.pkl')

Denoise path save in: /home/antslab/NAS1_RAID6/pcap_inter/2020_01_06/台灣大哥大/case_pickles/clusterName_overview_denoise_df_20200106_20200112.pkl


In [10]:
#注意參數!!第二次可以直接從這邊開始跑。
isp = '台灣大哥大'
date_li = ["20200106","20200107","20200108","20200109","20200110","20200111","20200112"]  #改!!
# date_li = ["20200106","20200107"]  #改!!
file_name = "_".join(sorted(date_li)) #date_li 改!!
min_date = str(min(date_li))
max_date = str(max(date_li))

In [11]:
#第二次這邊跑:
noise_path = '/home/antslab/NAS1_RAID6/pcap_inter/'+str(min_date[:4])+'_'+str(min_date[4:6])+'_'+str(min_date[6:])+'/'+isp+'/case_pickles/noise_cluster.pkl'
denoise_path = '/home/antslab/NAS1_RAID6/pcap_inter/'+str(min_date[:4])+'_'+str(min_date[4:6])+'_'+str(min_date[6:])+'/'+isp+'/case_pickles/denoise_cluster.pkl'
pickle_dir = '/home/antslab/NAS1_RAID6/pcap_inter/'+str(min_date[:4])+'_'+str(min_date[4:6])+'_'+str(min_date[6:])+'/'+isp+'/case_pickles/'
all_df = pickle.load(open(pickle_dir+'clusterName_overview_denoise_df_'+str(min_date)+'_'+str(date_li[-1])+'.pkl','rb'))
noise_clusters = pickle.load(open(noise_path,'rb'))
denoise_clusters = pickle.load(open(denoise_path,'rb'))
assert len(noise_clusters) + len(denoise_clusters) == len(all_df)
all_df = all_df[~all_df.idx.isin(noise_clusters)]
# all_df = all_df[all_df.idx.isin(denoise_clusters)]
# all_df = all_df.reset_index(drop=True)
all_df
# noise_clusters

,idx,timestamp,country,src_ip,ssdeep
0,20200106_23217_http,"[1578239430.609722, 1578239432.907151, 1578239...","[United States, United States, United States, ...","[66.249.64.245, 66.249.65.104, 66.249.71.53, 6...",[24:tA4ucXoajPt1HysrEA4s2XDCTjeN1dqgrFxtCBtDhL...
1,20200106_595_http,"[1578239430.767826, 1578239432.515023, 1578239...","[Iran, China, Belarus, China, China, Philippin...","[2.183.95.130, 49.234.29.65, 178.122.68.106, 4...",[12:JkHXq/IsheY1awh5UP+/4D7eh2x7AGGH46ZY1Zu8m:...
2,20200106_1700_http,"[1578239431.472793, 1578239434.482427, 1578239...","[Canada, Colombia, United States, Canada, Cana...","[172.105.4.227, 181.129.123.202, 216.244.66.24...",[12:rHXq6x7YNDIsqCjh8wHhFMsml6n61HCY3NkZfsv8m:...
3,20200106_249108104923_http,"[1578289690.645383, 1578302389.776345, 1578338...","[United States, China, China, Morocco, China, ...","[154.202.13.17, 111.230.23.22, 49.234.96.87, 1...",[12:7KS/cHnMQ7s95HfSBBBBBBBBBBBBBSjmHXq/IYDr+b...
6,20200106_1440_http,"[1578239428.028117, 1578239433.566138, 1578239...","[Russia, United States, Kazakhstan, Belarus, U...","[46.146.126.112, 66.249.71.108, 85.29.166.247,...",[24:tAzfGCU6BRCTS18YH+w18YHYR385Hltb17KAzfGCU6...
...,...,...,...,...,...
822,20200112_24491_telnet,"[1578789078.671959, 1578800410.996216]","[United States, United States]","[204.93.154.214, 204.93.154.211]","[3:ro53fg5T:ro2N, 3:ro53fg5T:ro2N, 3:ro53fg5T:..."
823,20200110_688_sip,"[1578591003.982135, 1578591003.997796, 1578591...","[France, France, France, France, France, Franc...","[5.135.73.105, 5.135.73.105, 5.135.73.105, 5.1...",[24:pII7mDETT+c715k35PUkTR3liRQSi3H5Ki6T+c0b3Z...
824,20200106_13657_sip,"[1578314626.450293, 1578314626.453001, 1578314...","[France, France, France, France, France, Franc...","[5.135.73.105, 5.135.73.105, 5.135.73.105, 5.1...",[48:0aauuILzzp+7aue08oKY18Vje821BCkOPk/Ix:Lfus...
825,20200106_1589_sip,"[1578240132.276525, 1578240205.46512, 15782402...","[United States, United States, United States, ...","[104.206.128.26, 104.206.128.14, 104.206.128.7...",[24:qgOf3vI7mDL5gizTKiqf3vX4v/DRt0UUgbYMfy1wUX...


In [12]:
def cut_protocol(cluster_id):
    '''
    GOAL: 將cluster index轉換為protocol名稱
    '''
    return cluster_id.split("_")[-1]
def count_sessions_num(time_li):
    '''
    GOAL: 依據timestamp list計算每個cluster所具有的session數量
    '''
    return len(time_li)
all_df['protocol'] = all_df.idx.map(cut_protocol)
all_df['session_num'] = all_df.timestamp.map(count_sessions_num)
print(isp,"經濾除noiseClusters後在",min_date,"至",max_date,"的期間中各protocols的session數量:")
#all_df.groupby("protocol")['timestamp'].sum().apply(len) #alternative
temp = pd.DataFrame(all_df.groupby('protocol')['session_num'].sum()).sort_values('session_num',ascending=False)
temp

台灣大哥大 經濾除noiseClusters後在 20200106 至 20200112 的期間中各protocols的session數量:


,session_num
protocol,
ssh,5680920
smb,884687
http,664096
smtp,56905
sip,25683
ftp,24733
rpc,8679
pop,3895
mysql,2757


In [13]:
#find all ips
all_ips = all_df.src_ip.tolist()
all_ips = sum(all_ips,[])
all_ips = list(set(all_ips))
all_ips = sorted(all_ips)
# col_li = ['Country']
# col_li.extend(all_df.idx.tolist())
col_li = all_df.idx.tolist()
# col_li = [x for x in col_li if 'tds' not in x]
# col_li.append('tds') #改!!
jc_matrix = pd.DataFrame(0, index=all_ips, columns=col_li)
col_li

['20200106_23217_http',
 '20200106_595_http',
 '20200106_1700_http',
 '20200106_249108104923_http',
 '20200106_1440_http',
 '20200106_7465_http',
 '20200106_1846_http',
 '20200106_3315_http',
 '20200106_23657_http',
 '20200106_3180_http',
 '20200106_3314_http',
 '20200106_2190_http',
 '20200106_13357_http',
 '20200106_1587_http',
 '20200106_8589936534_http',
 '20200106_28010_http',
 '20200106_2877_http',
 '20200106_2928_http',
 '20200106_34359764680_http',
 '20200106_14544_http',
 '20200106_154618860649_http',
 '20200106_13395_http',
 '20200106_8589941939_http',
 '20200106_3309_http',
 '20200106_24009_http',
 '20200106_1708_http',
 '20200106_1633_http',
 '20200106_27985_http',
 '20200106_427_http',
 '20200106_9955_http',
 '20200106_23675_http',
 '20200106_13437_http',
 '20200107_6299_http',
 '20200106_8589969169_http',
 '20200106_27999_http',
 '20200106_39176_http',
 '20200106_8545_http',
 '20200106_34359775846_http',
 '20200106_8589936509_http',
 '20200106_51539631350_http',
 '2020010

In [14]:
for col in tqdm(jc_matrix.columns.tolist()):
    if 'tds' in col: # 統一tds欄位，如果有的話
        select_df = all_df[all_df.idx.str.contains('tds')]
        ip_li = list(select_df.src_ip.values)
        try:
            for ips in ip_li:
                jc_matrix.loc[ips,'tds'] = 1
#             jc_matrix = jc_matrix.drop([col])
        except IndexError:
            print("Didn't load tds protocol to all_ip. SKIPPING!")
            pass
    else:
        select_df = all_df[all_df.idx == col]
        ip_li = list(select_df.src_ip.values)
        for ips in ip_li:      
            jc_matrix.loc[ips,col] = 1
# jc_matrix['np_array'] = list(jc_matrix.values)#.ravel()
jc_matrix

100%|██████████| 608/608 [00:03<00:00, 198.07it/s]


,20200106_23217_http,20200106_595_http,20200106_1700_http,20200106_249108104923_http,20200106_1440_http,20200106_7465_http,20200106_1846_http,20200106_3315_http,20200106_23657_http,20200106_3180_http,...,20200111_77309424108_telnet,20200107_8589999176_telnet,20200111_42949694294_telnet,20200111_77309442230_telnet,20200111_180388628604_telnet,20200112_24491_telnet,20200110_688_sip,20200106_13657_sip,20200106_1589_sip,20200106_26617_sip
1.0.134.96,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1.0.146.176,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1.0.153.160,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1.0.170.232,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1.0.192.198,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99.89.116.178,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
99.89.92.55,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
99.95.141.116,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
99.95.141.225,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
jc_matrix_stat = jc_matrix.append(pd.Series(jc_matrix.sum(),name='stat'))
jc_matrix_stat['np_array'] = list(jc_matrix_stat.values)
def sum_arr(npy):
    return sum(npy)
jc_matrix_stat['sum'] = jc_matrix_stat.np_array.apply(sum_arr)
jc_matrix_stat

,20200106_23217_http,20200106_595_http,20200106_1700_http,20200106_249108104923_http,20200106_1440_http,20200106_7465_http,20200106_1846_http,20200106_3315_http,20200106_23657_http,20200106_3180_http,...,20200111_42949694294_telnet,20200111_77309442230_telnet,20200111_180388628604_telnet,20200112_24491_telnet,20200110_688_sip,20200106_13657_sip,20200106_1589_sip,20200106_26617_sip,np_array,sum
1.0.134.96,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
1.0.146.176,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
1.0.153.160,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
1.0.170.232,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
1.0.192.198,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99.89.92.55,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",2
99.95.141.116,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
99.95.141.225,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
99.99.139.67,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...",1


In [16]:
jc_matrix_stat = jc_matrix_stat.drop(['np_array'],axis=1)
# pickle_dir = '/home/antslab/spark_data/pcap_inter/2020_01_06/中華電信/case_pickles/'
pickle.dump(file=open(pickle_dir+'clusters_ips_stat_df_'+str(min_date)+'_'+str(max_date)+'.pkl','wb'),obj=jc_matrix_stat)
print('One hot統計df:',pickle_dir+'clusters_ips_stat_df_'+str(min_date)+'_'+str(max_date)+'.pkl')

One hot統計df: /home/antslab/NAS1_RAID6/pcap_inter/2020_01_06/台灣大哥大/case_pickles/clusters_ips_stat_df_20200106_20200112.pkl


In [17]:
#濾除col:
jc_matrix_new = jc_matrix.append(pd.Series(jc_matrix.sum(),name='stat'))
col_need = []
for col in jc_matrix_new.columns:
    if col == 'np_array':
        continue
    if jc_matrix_new.loc['stat',col] > 1:
        col_need.append(col)
jc_matrix_new = jc_matrix_new[col_need]
#濾除row:
jc_matrix_new['np_array'] = list(jc_matrix_new.values)#.ravel()
jc_matrix_new['sum'] = jc_matrix_new.np_array.apply(sum_arr)
#要存下統計變量!刪掉np_array
jc_matrix_new = jc_matrix_new[jc_matrix_new['sum']>1]
jc_matrix_new
# jc_matrix.head(10).to_excel('/home/antslab/spark_data/pcap_inter/2020_01_06/中華電信/case_pickles/clique/picture/one-hot.xlsx')

,20200106_23217_http,20200106_595_http,20200106_1700_http,20200106_249108104923_http,20200106_1440_http,20200106_7465_http,20200106_1846_http,20200106_3315_http,20200106_23657_http,20200106_3180_http,...,20200111_2230_telnet,20200110_12842_telnet,20200111_77309424108_telnet,20200111_42949694294_telnet,20200112_24491_telnet,20200106_13657_sip,20200106_1589_sip,20200106_26617_sip,np_array,sum
1.10.186.35,0,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"[0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",2
1.10.188.203,0,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"[0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",2
1.119.197.134,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",3
1.161.238.229,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",2
1.161.91.244,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99.254.114.95,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",3
99.58.200.77,0,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"[0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",2
99.75.160.188,0,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",2
99.89.92.55,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",2


In [18]:
jc_matrix = jc_matrix_new.iloc[0:len(jc_matrix_new)-1]
jc_matrix = jc_matrix.sort_values(['sum'],ascending=False)
jc_matrix = jc_matrix.drop(['sum','np_array'],axis=1)
jc_matrix['np_array'] = list(jc_matrix.values)
jc_matrix

,20200106_23217_http,20200106_595_http,20200106_1700_http,20200106_249108104923_http,20200106_1440_http,20200106_7465_http,20200106_1846_http,20200106_3315_http,20200106_23657_http,20200106_3180_http,...,20200106_48295_telnet,20200111_2230_telnet,20200110_12842_telnet,20200111_77309424108_telnet,20200111_42949694294_telnet,20200112_24491_telnet,20200106_13657_sip,20200106_1589_sip,20200106_26617_sip,np_array
5.182.39.94,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
5.188.86.167,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
5.188.86.168,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
5.188.87.49,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
211.95.50.7,0,1,1,0,0,1,1,0,0,0,...,0,1,0,1,1,0,1,0,0,"[0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188.240.208.142,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
188.242.71.212,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ..."
188.244.189.11,0,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
188.254.0.170,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [19]:
#檢查!! 不能有assertion err!!!
def sum_val(npy):
    return sum(npy)
jc_matrix['sum'] = jc_matrix['np_array'].apply(sum_val)
temp = jc_matrix[jc_matrix['sum'] == 0]
weird_ips = temp.index.tolist()
assert len(weird_ips) == 0
jc_matrix = jc_matrix.drop(['sum'],axis=1)
gc.collect()

53

In [20]:
jc_matrix_stat = jc_matrix.append(pd.Series(jc_matrix.sum(),name='stat'))
# jc_matrix_stat['np_array'] = list(jc_matrix_stat.values)
def sum_arr(npy):
    return sum(npy)
jc_matrix_stat['sum'] = jc_matrix_stat.np_array.apply(sum_arr)
jc_matrix_stat = jc_matrix_stat.drop(['np_array'],axis=1)
# pickle_dir = '/home/antslab/spark_data/pcap_inter/2020_01_06/中華電信/case_pickles/'
pickle.dump(file=open(pickle_dir+'clusters_ips_stat_afterFilter_df_'+str(min_date)+'_'+str(max_date)+'.pkl','wb'),obj=jc_matrix_stat)
print("把col=1,row=1以下的濾掉之統計df:",pickle_dir+'clusters_ips_stat_afterFilter_df_'+str(min_date)+'_'+str(max_date)+'.pkl')
jc_matrix_stat

把col=1,row=1以下的濾掉之統計df: /home/antslab/NAS1_RAID6/pcap_inter/2020_01_06/台灣大哥大/case_pickles/clusters_ips_stat_afterFilter_df_20200106_20200112.pkl


,20200106_23217_http,20200106_595_http,20200106_1700_http,20200106_249108104923_http,20200106_1440_http,20200106_7465_http,20200106_1846_http,20200106_3315_http,20200106_23657_http,20200106_3180_http,...,20200106_48295_telnet,20200111_2230_telnet,20200110_12842_telnet,20200111_77309424108_telnet,20200111_42949694294_telnet,20200112_24491_telnet,20200106_13657_sip,20200106_1589_sip,20200106_26617_sip,sum
5.182.39.94,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,60
5.188.86.167,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,52
5.188.86.168,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,51
5.188.87.49,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,47
211.95.50.7,0,1,1,0,0,1,1,0,0,0,...,0,1,0,1,1,0,1,0,0,47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188.242.71.212,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
188.244.189.11,0,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
188.254.0.170,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
99.89.92.55,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2


In [ ]:
jc_matrix_stat[jc_matrix_stat['sum']<2] #要0 rows

In [21]:
def calc_jac(c_value,t_value):
    '''
    GOAL: 同時考量jaccrd計算方式，與人類直覺計算方式
    '''
    j_s = jaccard_score(c_value, t_value)
    c_s = cosine_similarity([c_value], [t_value])[0][0]
    one_portion = max(sum(c_value),sum(t_value))/len(t_value) #最大長度的1的數量
    final_score = (c_s*one_portion)+(j_s*(1-one_portion))
    return final_score
#     return jaccard_score(c_value, t_value)

def calc_cos(c_value,t_value):
    return cosine_similarity(c_value, t_value)

In [ ]:
#koalas
# jc_matrix2 = jc_matrix.drop(['np_array'],axis=1)
# jc_matrix = ks.from_pandas(jc_matrix2)

In [22]:
# 不同thr都會需要重跑一次
gc.collect()
thr_li= [0.1,0.5,0.9] #,0.8,0.7,0.6,0.5,0.4,0.3,0.2
# thr_li = [0.1,0.5]
for thr in tqdm(thr_li):
    jc_dict = {}
    ip_li = jc_matrix.index.tolist() #pandas
#     ip_li = list(jc_matrix.index.to_numpy()) #koalas
    used_ip = []
    for ip in ip_li:
        if ip in used_ip: #合併過得拿掉 single label
            continue
        t_value = jc_matrix.loc[ip,'np_array']# pandas
#         t_value = jc_matrix.loc[ip].values #koalas
        jc_calc = jc_matrix[~jc_matrix.index.isin(used_ip)] #合併過得拿掉 single label
        jc_calc = jc_calc[jc_calc.index != ip] #自己的不比 singleLabel
    #     jc_calc = jc_matrix[jc_matrix.index != ip] #自己的不比    multilabel
        jc_calc['jc_score'] = jc_calc.np_array.apply(calc_jac,args=(t_value,)) #得到t跟每個c的分數
    #     jc_calc['co_score'] = jc_calc.np_array.apply(calc_cos,args=(t_value,)) #得到t跟每個c的分數
        combine_df = jc_calc[jc_calc['jc_score']>thr] #所設定的相似度分數
        c_ips_li = combine_df.index.tolist() #跟這個IP具高度相似度的IPs
        if len(c_ips_li)>0:
            jc_dict[ip] = c_ips_li
            used_ip.extend(c_ips_li) #合併過的不要再比
            used_ip.append(ip) #用過的不要再比

    loner_ip = list(set(ip_li) - set(used_ip))
    min_date = str(min(date_li))
    pickle_path = pickle_dir+'CorrelateIP_APTIP_thr'+str(thr)+'_'+file_name+'.pkl' #改!!
    pickle.dump(obj=(jc_dict,loner_ip),file=open(pickle_path,'wb'))
    print("threshold =",thr,"(jaccard_dictionary,loner_ip) save path:",pickle_path)
    print('集團數量(IP>1,score>'+str(thr)+'):',len(jc_dict),"LonerIP數量:",len(loner_ip))

#thr=0.9: 集團數量(IP>1,score>thr): 996 LonerIP數量: 1897
#thr=0.8: 集團數量(IP>1,score>thr): 1005 LonerIP數量: 1566

 33%|███▎      | 1/3 [01:19<02:39, 79.66s/it]

threshold = 0.1 (jaccard_dictionary,loner_ip) save path: /home/antslab/NAS1_RAID6/pcap_inter/2020_01_06/台灣大哥大/case_pickles/CorrelateIP_APTIP_thr0.1_20200106_20200107_20200108_20200109_20200110_20200111_20200112.pkl
集團數量(IP>1,score>0.1): 25 LonerIP數量: 4


 67%|██████▋   | 2/3 [24:59<08:01, 481.77s/it]

threshold = 0.5 (jaccard_dictionary,loner_ip) save path: /home/antslab/NAS1_RAID6/pcap_inter/2020_01_06/台灣大哥大/case_pickles/CorrelateIP_APTIP_thr0.5_20200106_20200107_20200108_20200109_20200110_20200111_20200112.pkl
集團數量(IP>1,score>0.5): 197 LonerIP數量: 126


100%|██████████| 3/3 [2:54:24<00:00, 3488.23s/it]

threshold = 0.9 (jaccard_dictionary,loner_ip) save path: /home/antslab/NAS1_RAID6/pcap_inter/2020_01_06/台灣大哥大/case_pickles/CorrelateIP_APTIP_thr0.9_20200106_20200107_20200108_20200109_20200110_20200111_20200112.pkl
集團數量(IP>1,score>0.9): 365 LonerIP數量: 593


In [ ]:
# one_portion = max(sum(c_value),sum(t_value))/len(t_value)
# j_s = jaccard_score(c_value,t_value)

# c_s = cosine_similarity([c_value],[t_value])[0][0]
# (c_s*one_portion)+(j_s*(1-one_portion))

In [ ]:
# %%time
# # multi label multi processing
# from multiprocessing import Manager, Pool
# thr=0.9
# ip_li = jc_matrix.index.tolist()

# manager = Manager()
# used_ip = manager.list()
# jc_dict = manager.dict()
# def do_ip(ip):
#     global used_ip
#     global jc_dict
#     global jc_matrix
# #     if ip in used_ip: #合併過得拿掉
# #         continue
#     t_value = jc_matrix.loc[ip,'np_array']#.values
# #     jc_calc = jc_matrix[~jc_matrix.index.isin(used_ip)] #合併過得拿掉
# #     jc_calc = jc_calc[jc_calc.index != ip] #自己的不比
#     jc_calc = jc_matrix[jc_matrix.index != ip] #自己的不比    
#     jc_calc['jc_score'] = jc_calc.np_array.apply(calc_jac,args=(t_value,)) #得到t跟每個c的分數
#     combine_df = jc_calc[jc_calc['jc_score']>thr] #所設定的相似度分數
#     c_ips_li = combine_df.index.tolist() #跟這個IP具高度相似度的IPs
#     if len(c_ips_li)>0:
#         jc_dict[ip] = c_ips_li
#         used_ip.extend(c_ips_li) #合併過的不要再比
#         used_ip.append(ip) #用過的不要再比

# pool = Pool(processes=12)
# pool.map(do_ip, ip_li)
# pool.close()


# loner_ip = list(set(ip_li) - set(used_ip))
# pickle_path = '/home/antslab/spark_data/pcap_inter/2020_01_'+str(min(date_li)[-2:])+'/中華電信/case_pickles/'+'CorrelateIP_APTIP_thr'+str(thr)+'_MultiLabels'+file_name+'.pkl'
# pickle.dump(obj=(jc_dict,loner_ip),file=open(pickle_path,'wb'))
# print('集團數量(IP>1,score>'+str(thr)+'):',len(jc_dict),"LonerIP數量:",len(loner_ip))


In [23]:
def find(myList,target ):
    return [i for i,j in enumerate(myList) if j == target]
def find_time(indexes,li):
    '''
    一個IP回傳一個time list
    '''
    return list(map(li.__getitem__, indexes))
def find_country(need_index,candidate_li):
    '''
    一個IP只回傳一個country
    '''
    return candidate_li[need_index[0]]
def repeat_idx(ori_li,index):
    return [index]*len(ori_li)

In [ ]:
# import more_itertools as mit
# from timeit import timeit

# data = list(range(1000))
# data.extend([1,2,1,1])
# def find_iter(data,target):
#     return list(mit.locate(data, lambda x: x == target))
# value=1
# ttt=2000
# t1 = timeit(lambda: find_iter(data,value), number=ttt)
# t2 = timeit(lambda: find(data, value), number=ttt)
# t1,t2

In [ ]:
%%time
# 綜合所需的表，不同thr只要跑一次 [增加country資訊] [增加loner可使用]
jc_matrix2 = jc_matrix.drop(['np_array'],axis=1)
ori_col = all_df.idx.tolist()
tds_ori_col = []
for col in ori_col:
    if 'tds' in col:
        tds_ori_col.append(col)
def find2(t_ip): #同個target ip
    global temp
    global temp2
    global t_idx
    temp = jc_matrix2[jc_matrix2.index == t_ip]
    t_idx = temp.columns[temp.eq(1).any()]
#     if 'tds' in t_idx:
#         t_idx = list(t_idx)
#         t_idx.extend(tds_ori_col)
    temp2 = all_df[all_df.idx.isin(t_idx)]
    temp2['gen'] = temp2.src_ip.apply(find,args=(t_ip,))
    temp2['time_li'] = temp2.apply(lambda x: find_time(x.gen, x.timestamp), axis=1)
    temp2['idx_li'] = temp2.apply(lambda x: repeat_idx(x.time_li, x.idx), axis=1)
    temp2['country'] =  temp2.apply(lambda x: find_country(x.gen, x.country), axis=1)
#     return temp2['time_li'].tolist() #list of lists
    return functools.reduce(operator.iconcat, temp2['time_li'].tolist(), []),functools.reduce(operator.iconcat, temp2['idx_li'].tolist(), []), temp2['country'].iloc[0] 

ip_li = jc_matrix.index.tolist()
ip_df = pd.DataFrame(ip_li,columns=['src_ip'])
ip_df['session_timelist'],ip_df['session_idlist'],ip_df['session_county'] = zip(*ip_df['src_ip'].apply(find2))
pickle.dump(obj=ip_df,file=open(pickle_dir+'CorrelateIP_ALL_ip_df.pkl','wb'))
# ip_df

In [ ]:
ip_df

In [ ]:
# #TRAIN SAVE:
# min_date = str(min(date_li))
# # pickle_dir = '/home/antslab/spark_data/pcap_inter/'+str(min_date[:4])+'_'+str(min_date[4:6])+'_'+str(min_date[6:])+'/中華電信/case_pickles/'  
# pickle.dump(obj=ip_df,file=open(pickle_dir+'CorrelateIP_ALL_ip_df.pkl','wb')) #改?

INFERENCE
* 可直接跑

In [25]:
date_li = ["20200106","20200107","20200108","20200109","20200110","20200111","20200112"] 
# date_li = ["20200106","20200107"] 
file_name = "_".join(sorted(date_li)) #date_li 改!!
thr_li = [0.1,0.5,0.9] #最後的值可以跑後面幾格的statistics
# thr_li = [0.5,0.9]

min_date = str(min(date_li))
pickle_dir = '/home/antslab/NAS1_RAID6/pcap_inter/'+str(min_date[:4])+'_'+str(min_date[4:6])+'_'+str(min_date[6:])+'/'+isp+'/case_pickles/'
ip_df = pickle.load(open(pickle_dir+'CorrelateIP_ALL_ip_df.pkl','rb')) #改?

for thr in tqdm(thr_li):
    # 改!!
    pickle_path = pickle_dir+'CorrelateIP_APTIP_thr'+str(thr)+'_'+file_name+'.pkl' #改!!
    # pickle_path = '/home/antslab/spark_data/pcap_inter/2020_01_06/中華電信/case_pickles/'+'CorrelateIP_APTIP_thr0.8_0106_0107_0108_0109_0110_0111_0112.pkl'
    jc_dict,loner_ip = pickle.load(open(pickle_path,'rb'))

    similarity_id_list = []
    timelist_dict_list = []
    clusterlist_dict_list = []
    # country_dict = []
    country_list = []

    for cluster_id, ip_li in jc_dict.items():
        all_ips = ip_li[:]
        all_ips.append(cluster_id)
        temp = ip_df[ip_df.src_ip.isin(all_ips)]
        temp_time = temp.set_index('src_ip')['session_timelist'].to_dict()
        temp_id = temp.set_index('src_ip')['session_idlist'].to_dict()
    #     temp_country = temp.set_index('src_ip')['country'].to_dict()
        temp_country = temp['session_county'].tolist()
        similarity_id_list.append(cluster_id) #僅識別用
        timelist_dict_list.append(temp_time)
        clusterlist_dict_list.append(temp_id)
        country_list.append(temp_country)
    pattern_select_df = pd.DataFrame([similarity_id_list,timelist_dict_list,clusterlist_dict_list,country_list],
                 index=['pattern_key','sessions_time_dict','cluster_id_dict','country_list']).T
    save_path = pickle_dir+'CorrelateIP_DRAW_'+str(thr)+'.pkl'
    pickle.dump(obj=pattern_select_df,file=open(save_path,'wb'))
    print("集團數量(IP>1,score>"+str(thr)+"):",len(jc_dict),"LonerIP數量:",len(loner_ip))
    print('視覺化路徑:',save_path)
#     print(pattern_select_df)
pattern_select_df

 33%|███▎      | 1/3 [00:00<00:00,  2.16it/s]

集團數量(IP>1,score>0.1): 25 LonerIP數量: 4
視覺化路徑: /home/antslab/NAS1_RAID6/pcap_inter/2020_01_06/台灣大哥大/case_pickles/CorrelateIP_DRAW_0.1.pkl


 67%|██████▋   | 2/3 [00:00<00:00,  2.21it/s]

集團數量(IP>1,score>0.5): 197 LonerIP數量: 126
視覺化路徑: /home/antslab/NAS1_RAID6/pcap_inter/2020_01_06/台灣大哥大/case_pickles/CorrelateIP_DRAW_0.5.pkl


100%|██████████| 3/3 [00:01<00:00,  1.96it/s]

集團數量(IP>1,score>0.9): 365 LonerIP數量: 593
視覺化路徑: /home/antslab/NAS1_RAID6/pcap_inter/2020_01_06/台灣大哥大/case_pickles/CorrelateIP_DRAW_0.9.pkl


,pattern_key,sessions_time_dict,cluster_id_dict,country_list
0,107.6.183.162,"{'107.6.183.162': [1578245255.380018, 15782600...","{'107.6.183.162': ['20200106_595_http', '20200...","[Netherlands, Netherlands]"
1,198.20.103.242,"{'198.20.103.242': [1578256883.688391, 1578256...","{'198.20.103.242': ['20200106_595_http', '2020...","[Netherlands, United States]"
2,107.6.169.250,"{'107.6.169.250': [1578241964.465474, 15782429...","{'107.6.169.250': ['20200106_595_http', '20200...","[Netherlands, United States, United States]"
3,185.100.87.191,"{'185.100.87.191': [1578330424.780331, 1578330...","{'185.100.87.191': ['20200106_595_http', '2020...","[Romania, United States]"
4,185.100.87.245,"{'185.100.87.245': [1578330728.879811, 1578330...","{'185.100.87.245': ['20200106_595_http', '2020...","[Romania, United States, Romania, Romania]"
...,...,...,...,...
360,211.38.159.129,"{'211.38.159.129': [1578270324.174275, 1578796...","{'211.38.159.129': ['20200106_1610_ftp', '2020...","[South Korea, Thailand, China]"
361,196.52.43.119,"{'196.52.43.119': [1578601951.741461, 15787056...","{'196.52.43.119': ['20200106_595_http', '20200...","[South Africa, Albania]"
362,45.143.222.191,"{'45.143.222.191': [1578239432.430883, 1578239...","{'45.143.222.191': ['20200106_622_smtp', '2020...","[Netherlands, Netherlands, China, China, China..."
363,36.37.73.244,"{'36.37.73.244': [1578291583.783071, 157829588...","{'36.37.73.244': ['20200106_1440_http', '20200...","[Indonesia, Colombia]"


#### Statistics

找IP在哪裡

In [ ]:
def find_ip(di):
    if '194.61.24.75' in di.keys():
        return True
    else:
        return False

In [ ]:
temp = pattern_df[:]
temp['isin'] = temp.sessions_time_dict.map(find_ip)
temp = temp[temp['isin'] == True]
temp

欄列統計資料

In [26]:
#有多少column只有1 (這個行為只有一個人做)
column_df = jc_matrix.drop(['np_array'],axis='columns')
column_df = column_df.sum(axis='index')
column_df = pd.DataFrame(column_df)
column_df = column_df.sort_values(0,ascending=False)
column_df

,0
20200107_6_ssh,8701
20200106_291_ssh,7689
20200112_216_ssh,7289
20200106_1700_http,5643
20200106_595_http,5077
...,...
20200110_12842_telnet,0
20200106_558345772574_smtp,0
20200112_24491_telnet,0
20200106_34359764680_http,0


In [27]:
col_draw = dict(Counter(column_df[0].tolist()))
col_draw = dict(sorted(col_draw.items()))
file_name = pickle_dir+'BehaviorCluster_ips_statistics'+str(thr)+'_'+str(min_date)+'_'+str(max_date)+'.pkl' #改!!
pickle.dump(file=open(file_name,'wb'),obj=(column_df,col_draw)) #(df,dict)
print("col statistics tuple SAVE IN:",file_name)

col statistics tuple SAVE IN: /home/antslab/NAS1_RAID6/pcap_inter/2020_01_06/台灣大哥大/case_pickles/BehaviorCluster_ips_statistics0.9_20200106_20200112.pkl


In [28]:
# 有多少row只有1 (這個人只做一件事情)
row_df = jc_matrix.drop(['np_array'],axis='columns')
row_df = row_df.sum(axis='columns')
row_df = pd.DataFrame(row_df)
row_df = row_df.sort_values(0,ascending=False)
row_df

,0
5.182.39.94,60
5.188.86.167,52
5.188.86.168,51
211.95.50.7,47
5.188.87.49,47
...,...
139.5.222.106,2
139.255.91.58,2
139.255.87.213,2
139.255.131.12,2


In [29]:
row_draw = dict(Counter(row_df[0].tolist()))
row_draw = dict(sorted(row_draw.items()))
file_name = pickle_dir+'ip_behaviorCluster_statistics'+str(thr)+'_'+str(min_date)+'_'+str(max_date)+'.pkl' #改!!
pickle.dump(file=open(file_name,'wb'),obj=(row_df,row_draw)) #(df,dict)
print("row statistics SAVE IN:",file_name)

row statistics SAVE IN: /home/antslab/NAS1_RAID6/pcap_inter/2020_01_06/台灣大哥大/case_pickles/ip_behaviorCluster_statistics0.9_20200106_20200112.pkl


In [ ]:
row_df[row_df[0] >=100]

In [30]:
#每個IP有多少個session?
global ip_session_count 
ip_session_count = {}
def find_sessions_number(ip_li):
    '''
    INPUT: list
    
    '''
    global ip_session_count
    for ip in ip_li:
        try:
            val = ip_session_count[ip]
            ip_session_count[ip] = val+1
        except:
            ip_session_count[ip] = 1
all_df.src_ip.apply(find_sessions_number)
ip_session_df = pd.DataFrame(ip_session_count.items())
print("Total sessions#:",ip_session_df[1].sum())
ip_session_df

Total sessions#: 7355454


,0,1
0,66.249.64.245,78
1,66.249.65.104,29
2,66.249.71.53,310
3,66.249.71.111,299
4,66.249.64.83,112
...,...,...
84863,34.220.212.39,1
84864,54.193.91.151,1
84865,204.93.154.214,1
84866,204.93.154.211,1


In [31]:
#有m個session(key)的IP有幾個(value)
ip_draw = dict(Counter(ip_session_df[1].tolist()))
ip_draw = dict(sorted(ip_draw.items()))
file_name = pickle_dir+'ip_sessions_statistics'+str(thr)+'.pkl' #改!!
pickle.dump(file=open(file_name,'wb'),obj=ip_draw)
print("有m個session(key)的IP有幾個(value) SAVE IN:",file_name)

有m個session(key)的IP有幾個(value) SAVE IN: /home/antslab/NAS1_RAID6/pcap_inter/2020_01_06/台灣大哥大/case_pickles/ip_sessions_statistics0.9.pkl


loner ip 所對應的 cluster name

In [32]:
loner_cluster_dict = {}
jc_matrix2 = jc_matrix.drop(['np_array'],axis='columns')
for ip in loner_ip:
    temp = pd.DataFrame(jc_matrix2.loc[ip])
    temp = temp[temp[ip]==1]
    cluster_name_li = temp.index.tolist()
    loner_cluster_dict[ip] = cluster_name_li
print("LonerIP共涵蓋",len(loner_cluster_dict),"個clusters")
 
loner_cluster_df = pd.DataFrame(loner_cluster_dict.items())
loner_cluster_df[2] = loner_cluster_df[1].map(len)
loner_cluster_df.columns = ['src_ip','cluster_name','cluster_num']
file_name = pickle_dir+'lonerip_clusterName_df'+str(thr)+'.pkl' #改!!
pickle.dump(file=open(file_name,'wb'),obj=loner_cluster_df)
print("loner ip 所對應的 cluster name df SAVE IN:",file_name)   
loner_cluster_df

LonerIP共涵蓋 593 個clusters
loner ip 所對應的 cluster name df SAVE IN: /home/antslab/NAS1_RAID6/pcap_inter/2020_01_06/台灣大哥大/case_pickles/lonerip_clusterName_df0.9.pkl


,src_ip,cluster_name,cluster_num
0,5.188.86.178,"[20200106_291_ssh, 20200107_6_ssh, 20200112_21...",30
1,93.174.95.106,"[20200106_595_http, 20200106_1700_http, 202001...",17
2,195.154.237.102,"[20200106_1587_http, 20200106_27985_http]",2
3,23.19.58.161,"[20200106_622_smtp, 20200106_1588_smtp, 202001...",3
4,112.33.254.28,"[20200106_1610_ftp, 20200106_2131_ftp, 2020010...",4
...,...,...,...
588,172.105.75.101,"[20200106_1846_http, 20200106_427_http, 202001...",3
589,170.106.37.63,"[20200106_17179895470_pop, 20200109_5576_pop]",2
590,219.141.211.66,"[20200106_291_ssh, 20200106_33254_ssh, 2020010...",3
591,45.56.68.15,"[20200106_1846_http, 20200106_3309_http, 20200...",6


In [ ]:
loner_cluster_df.cluster_num.describe()

計算國家、IP數量、proto數量、cluster數量

In [ ]:
可不用跑
#pattern_select_df = pickle.load(open('/home/antslab/spark_data/pcap_inter/2020_01_06/中華電信/case_pickles/CorrelateIP_DRAW_0.1.pkl','rb'))
# pattern_select_df

In [ ]:
# Deprecated! Need Fix !!
cluster_name_dict = pickle.load(open('/home/antslab/NAS2_RAID5/pcap_inter/2020_01_06/中華電信/case_pickles/intention_dict_0106_0112.pkl','rb'))
print(cluster_name_dict.keys())
intention_dict = {}
for in_name,cluster_li in cluster_name_dict.items():
    for cluster in cluster_li:
        intention_dict[cluster] = in_name
print(set(intention_dict.values()))
intention_dict

In [33]:

def count_ip(di):
    '''
    GOAL: count ip number
    '''
    return len(di)
def count_cluster(di):
    '''
    GOAL: count unique clusters #
    '''
    li =  list(di.values())
    return len(set(functools.reduce(operator.iconcat, li, [])))
def proto_li(di):
    '''
    GOAL: extract protocols names
    Return: unique list
    '''
    li = list(di.values())
    cluster_li = list(set(functools.reduce(operator.iconcat, li, [])))
    proto_li = [x.split('_')[-1] for x in cluster_li]
    return sorted(set(proto_li))
def country_li(li):
    lis = list(set(li))
    lis = [str(x) for x in lis]
    return sorted(lis)
#     return sorted(list(set(li)))
def cluster_li(di):
    '''
    GOAL: extract clusters names
    Return: unique list
    '''
    li = list(di.values())
    cluster_li = list(set(functools.reduce(operator.iconcat, li, [])))
    return sorted(set(cluster_li))
def country_count(li):
    '''
    GOAL: count countries in the group's num
    '''
    count_dict = dict(Counter(li))
    return {k: v for k, v in sorted(count_dict.items(), key=lambda item: item[1],reverse=True)}
def country_portion(di):
    '''
    GOAL: count country's port ion the group
    '''
    all_nums = sum(list(di.values()))
    df = pd.DataFrame(di.items())
    df[1] = df[1]/all_nums
    return df.set_index(0)[1].to_dict()
def main_country(di):
    '''
    GOAL: return main country
    '''
    return list(di.keys())[0]
def cluster_number(di):
    '''
    GOAL: count cluster number in each group
    '''
    li =  list(di.values())
    count_dict = dict(Counter(list(functools.reduce(operator.iconcat, li, []))))
    return {k: v for k, v in sorted(count_dict.items(), key=lambda item: item[1],reverse=True)}
def cluster_portion(di):
    '''
    GOAL: use cluster_num to calculate cluster % in each group
    '''
    all_nums = sum(list(di.values()))
    df = pd.DataFrame(di.items())
    df[1] = df[1]/all_nums
    return df.set_index(0)[1].to_dict()
def intention_number(tmp_di):
    '''
    GOAL: transfer cluster name to intention categories.
    '''
    intention_num = {}
    for c_name,c_num in tmp_di.items():
        try:
            i_name = intention_dict[c_name]
        except KeyError:
            i_name = 'probing'
        try:
            ori_num = intention_num[i_name]
            intention_num[i_name] = ori_num + int(c_num)
        except KeyError:
            intention_num[i_name] = int(c_num)
    return {k: v for k, v in sorted(intention_num.items(), key=lambda item: item[1],reverse=True)}
def intention_portion(di):
    '''
    GOAL: calculate intention category's portion in dict type.
    '''
    all_nums = sum(list(di.values()))
    df = pd.DataFrame(di.items())
    df[1] = df[1]/all_nums
    return df.set_index(0)[1].to_dict()    

pattern_select_df['country_set'] = pattern_select_df.country_list.map(country_li)
pattern_select_df['country_nums'] = pattern_select_df['country_list'].map(country_count)
pattern_select_df['country_portion'] = pattern_select_df['country_nums'].map(country_portion)
pattern_select_df['main_country'] = pattern_select_df['country_nums'].map(main_country)
pattern_select_df['proto_set'] = pattern_select_df.cluster_id_dict.map(proto_li)
pattern_select_df['cluster_set'] = pattern_select_df.cluster_id_dict.map(cluster_li)
pattern_select_df['cluster_nums'] = pattern_select_df.cluster_id_dict.map(cluster_number)
pattern_select_df['cluster_portion'] = pattern_select_df.cluster_nums.map(cluster_portion)
# Deprecated! Need Fix!!
# pattern_select_df['intention_nums'] = pattern_select_df.cluster_nums.map(intention_number)
# pattern_select_df['intention_portion'] = pattern_select_df.intention_nums.map(intention_portion)
pattern_select_df['ip_num'] = pattern_select_df.cluster_id_dict.map(count_ip)
pattern_select_df['unique_country_num'] = pattern_select_df.country_set.map(count_ip)
pattern_select_df['unique_cluster_num'] = pattern_select_df.cluster_id_dict.map(count_cluster)
pattern_select_df['unique_protocols_num'] = pattern_select_df.proto_set.map(count_ip)
pattern_select_df

,pattern_key,sessions_time_dict,cluster_id_dict,country_list,country_set,country_nums,country_portion,main_country,proto_set,cluster_set,cluster_nums,cluster_portion,ip_num,unique_country_num,unique_cluster_num,unique_protocols_num
0,107.6.183.162,"{'107.6.183.162': [1578245255.380018, 15782600...","{'107.6.183.162': ['20200106_595_http', '20200...","[Netherlands, Netherlands]",[Netherlands],{'Netherlands': 2},{'Netherlands': 1.0},Netherlands,"[ftp, http, imap, pop, smtp, ssh, telnet]","[20200106_1586_ftp, 20200106_1649_pop, 2020010...","{'20200106_32160_imap': 90, '20200106_595_http...","{'20200106_32160_imap': 0.11952191235059761, '...",2,1,44,7
1,198.20.103.242,"{'198.20.103.242': [1578256883.688391, 1578256...","{'198.20.103.242': ['20200106_595_http', '2020...","[Netherlands, United States]","[Netherlands, United States]","{'Netherlands': 1, 'United States': 1}","{'Netherlands': 0.5, 'United States': 0.5}",Netherlands,"[ftp, http, imap, pop, smtp, ssh, telnet]","[20200106_1586_ftp, 20200106_1649_pop, 2020010...","{'20200106_595_http': 47, '20200106_2576983012...","{'20200106_595_http': 0.10779816513761468, '20...",2,2,40,7
2,107.6.169.250,"{'107.6.169.250': [1578241964.465474, 15782429...","{'107.6.169.250': ['20200106_595_http', '20200...","[Netherlands, United States, United States]","[Netherlands, United States]","{'United States': 2, 'Netherlands': 1}","{'United States': 0.6666666666666666, 'Netherl...",United States,"[ftp, http, imap, pop, smtp, ssh, telnet]","[20200106_1586_ftp, 20200106_1649_pop, 2020010...","{'20200106_25769830122_smtp': 60, '20200106_59...",{'20200106_25769830122_smtp': 0.08982035928143...,3,2,40,7
3,185.100.87.191,"{'185.100.87.191': [1578330424.780331, 1578330...","{'185.100.87.191': ['20200106_595_http', '2020...","[Romania, United States]","[Romania, United States]","{'Romania': 1, 'United States': 1}","{'Romania': 0.5, 'United States': 0.5}",Romania,"[http, imap, pop, smb, smtp]","[20200106_0_smb, 20200106_11116_smtp, 20200106...","{'20200106_1846_http': 184, '20200106_1700_htt...","{'20200106_1846_http': 0.30976430976430974, '2...",2,2,23,5
4,185.100.87.245,"{'185.100.87.245': [1578330728.879811, 1578330...","{'185.100.87.245': ['20200106_595_http', '2020...","[Romania, United States, Romania, Romania]","[Romania, United States]","{'Romania': 3, 'United States': 1}","{'Romania': 0.75, 'United States': 0.25}",Romania,"[http, smb, smtp]","[20200106_0_smb, 20200106_11116_smtp, 20200106...","{'20200106_1846_http': 373, '20200106_1700_htt...","{'20200106_1846_http': 0.2909516380655226, '20...",4,2,22,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,211.38.159.129,"{'211.38.159.129': [1578270324.174275, 1578796...","{'211.38.159.129': ['20200106_1610_ftp', '2020...","[South Korea, Thailand, China]","[China, South Korea, Thailand]","{'South Korea': 1, 'Thailand': 1, 'China': 1}","{'South Korea': 0.3333333333333333, 'Thailand'...",South Korea,[ftp],"[20200106_1610_ftp, 20200106_85899383096_ftp]","{'20200106_1610_ftp': 4, '20200106_85899383096...","{'20200106_1610_ftp': 0.5714285714285714, '202...",3,3,2,1
361,196.52.43.119,"{'196.52.43.119': [1578601951.741461, 15787056...","{'196.52.43.119': ['20200106_595_http', '20200...","[South Africa, Albania]","[Albania, South Africa]","{'South Africa': 1, 'Albania': 1}","{'South Africa': 0.5, 'Albania': 0.5}",South Africa,"[ftp, http]","[20200106_1586_ftp, 20200106_595_http]","{'20200106_595_http': 3, '20200106_1586_ftp': 3}","{'20200106_595_http': 0.5, '20200106_1586_ftp'...",2,2,2,2
362,45.143.222.191,"{'45.143.222.191': [1578239432.430883, 1578239...","{'45.143.222.191': ['20200106_622_smtp', '2020...","[Netherlands, Netherlands, China, China, China...","[China, Netherlands, Poland]","{'China': 3, 'Netherlands': 2, 'Poland': 1}","{'China': 0.5, 'Netherlands': 0.33333333333333...",China,[smtp],"[20200106_622_smtp, 20200106_8589962662_smtp]","{'20200106_622_smtp': 8252, '20200106_85899626...","{'20200106_622_smtp': 0.99457635289863

In [ ]:
pattern_select_df.describe()

In [34]:
# thr=0.1
# pickle_dir = '/home/antslab/spark_data/pcap_inter/2020_01_06/中華電信/case_pickles/'
save_path = pickle_dir+'CorrelateIP_DRAW_stat'+str(thr)+'.pkl'
pickle.dump(obj=pattern_select_df,file=open(save_path,'wb'))
print('IP群統計做圖用df路徑:',save_path)

IP群統計做圖用df路徑: /home/antslab/NAS1_RAID6/pcap_inter/2020_01_06/台灣大哥大/case_pickles/CorrelateIP_DRAW_stat0.9.pkl


loner ip's country

In [ ]:
#可不跑
thr = 0.9
pickle_path = '/home/antslab/spark_data/pcap_inter/'+str(min_date[:4])+'_'+str(min_date[4:6])+'_'+str(min_date[6:])+'/中華電信/case_pickles/'+'CorrelateIP_APTIP_thr'+str(thr)+'_'+file_name+'.pkl' #改!!
ip_df = pickle.load(open(pickle_dir+'CorrelateIP_ALL_ip_df.pkl','rb')) #改
loner_ip = pickle.load(open(pickle_path,'rb'))[1]
loner_ip

In [35]:
loner_country_info = ip_df[ip_df.src_ip.isin(loner_ip)]
loner_country_info = loner_country_info.reset_index(drop=True)
loner_country_info

,src_ip,session_timelist,session_idlist,session_county
0,5.182.39.94,"[1578239428.924645, 1578239429.14547, 15782394...","[20200106_291_ssh, 20200106_291_ssh, 20200106_...",Russia
1,5.188.86.167,"[1578239427.730637, 1578239428.060435, 1578239...","[20200106_291_ssh, 20200106_291_ssh, 20200106_...",Ireland
2,5.188.86.168,"[1578239427.042959, 1578239427.050201, 1578239...","[20200106_291_ssh, 20200106_291_ssh, 20200106_...",Ireland
3,5.188.87.49,"[1578239428.473573, 1578239428.547306, 1578239...","[20200106_291_ssh, 20200106_291_ssh, 20200106_...",Ireland
4,211.95.50.7,"[1578749110.865581, 1578749110.916418, 1578711...","[20200106_595_http, 20200106_595_http, 2020010...",China
...,...,...,...,...
588,192.42.116.13,"[1578296273.641001, 1578343334.200849, 1578343...","[20200106_1440_http, 20200106_77309434453_http...",None
589,195.154.237.102,"[1578301463.282097, 1578347912.249426]","[20200106_1587_http, 20200106_27985_http]",France
590,194.180.224.249,"[1578275816.117197, 1578275911.574547, 1578275...","[20200106_1846_http, 20200106_1846_http, 20200...",United States
591,189.30.210.143,"[1578775211.33794, 1578775211.328326]","[20200106_8589982593_http, 20200106_5153965623...",Brazil


In [ ]:
loner_country_info.session_county.value_counts()

In [ ]:
loner_country_info.session_county.value_counts().head(60)

In [36]:
pickle.dump(file=open(pickle_dir+'loner_draw_country'+str(thr)+'.pkl','wb'),obj=loner_country_info)
print("loner df資訊(可畫圖):",pickle_dir+'loner_draw_country'+str(thr)+'.pkl')
loner_country_info

loner df資訊(可畫圖): /home/antslab/NAS1_RAID6/pcap_inter/2020_01_06/台灣大哥大/case_pickles/loner_draw_country0.9.pkl


,src_ip,session_timelist,session_idlist,session_county
0,5.182.39.94,"[1578239428.924645, 1578239429.14547, 15782394...","[20200106_291_ssh, 20200106_291_ssh, 20200106_...",Russia
1,5.188.86.167,"[1578239427.730637, 1578239428.060435, 1578239...","[20200106_291_ssh, 20200106_291_ssh, 20200106_...",Ireland
2,5.188.86.168,"[1578239427.042959, 1578239427.050201, 1578239...","[20200106_291_ssh, 20200106_291_ssh, 20200106_...",Ireland
3,5.188.87.49,"[1578239428.473573, 1578239428.547306, 1578239...","[20200106_291_ssh, 20200106_291_ssh, 20200106_...",Ireland
4,211.95.50.7,"[1578749110.865581, 1578749110.916418, 1578711...","[20200106_595_http, 20200106_595_http, 2020010...",China
...,...,...,...,...
588,192.42.116.13,"[1578296273.641001, 1578343334.200849, 1578343...","[20200106_1440_http, 20200106_77309434453_http...",None
589,195.154.237.102,"[1578301463.282097, 1578347912.249426]","[20200106_1587_http, 20200106_27985_http]",France
590,194.180.224.249,"[1578275816.117197, 1578275911.574547, 1578275...","[20200106_1846_http, 20200106_1846_http, 20200...",United States
591,189.30.210.143,"[1578775211.33794, 1578775211.328326]","[20200106_8589982593_http, 20200106_5153965623...",Brazil


In [ ]:
loner_country_info[loner_country_info.session_county == "China"]

手法(clusters)出現在哪些group、出現次數頻率
* all_df搭配pattern_select_df

In [37]:
# pattern_select_df = pickle.load(open('/home/antslab/spark_data/pcap_inter/2020_01_06/中華電信/case_pickles/CorrelateIP_DRAW_stat0.9.pkl','rb'))
cluster_names_li = all_df.idx.tolist()
pattern_select_df

,pattern_key,sessions_time_dict,cluster_id_dict,country_list,country_set,country_nums,country_portion,main_country,proto_set,cluster_set,cluster_nums,cluster_portion,ip_num,unique_country_num,unique_cluster_num,unique_protocols_num
0,107.6.183.162,"{'107.6.183.162': [1578245255.380018, 15782600...","{'107.6.183.162': ['20200106_595_http', '20200...","[Netherlands, Netherlands]",[Netherlands],{'Netherlands': 2},{'Netherlands': 1.0},Netherlands,"[ftp, http, imap, pop, smtp, ssh, telnet]","[20200106_1586_ftp, 20200106_1649_pop, 2020010...","{'20200106_32160_imap': 90, '20200106_595_http...","{'20200106_32160_imap': 0.11952191235059761, '...",2,1,44,7
1,198.20.103.242,"{'198.20.103.242': [1578256883.688391, 1578256...","{'198.20.103.242': ['20200106_595_http', '2020...","[Netherlands, United States]","[Netherlands, United States]","{'Netherlands': 1, 'United States': 1}","{'Netherlands': 0.5, 'United States': 0.5}",Netherlands,"[ftp, http, imap, pop, smtp, ssh, telnet]","[20200106_1586_ftp, 20200106_1649_pop, 2020010...","{'20200106_595_http': 47, '20200106_2576983012...","{'20200106_595_http': 0.10779816513761468, '20...",2,2,40,7
2,107.6.169.250,"{'107.6.169.250': [1578241964.465474, 15782429...","{'107.6.169.250': ['20200106_595_http', '20200...","[Netherlands, United States, United States]","[Netherlands, United States]","{'United States': 2, 'Netherlands': 1}","{'United States': 0.6666666666666666, 'Netherl...",United States,"[ftp, http, imap, pop, smtp, ssh, telnet]","[20200106_1586_ftp, 20200106_1649_pop, 2020010...","{'20200106_25769830122_smtp': 60, '20200106_59...",{'20200106_25769830122_smtp': 0.08982035928143...,3,2,40,7
3,185.100.87.191,"{'185.100.87.191': [1578330424.780331, 1578330...","{'185.100.87.191': ['20200106_595_http', '2020...","[Romania, United States]","[Romania, United States]","{'Romania': 1, 'United States': 1}","{'Romania': 0.5, 'United States': 0.5}",Romania,"[http, imap, pop, smb, smtp]","[20200106_0_smb, 20200106_11116_smtp, 20200106...","{'20200106_1846_http': 184, '20200106_1700_htt...","{'20200106_1846_http': 0.30976430976430974, '2...",2,2,23,5
4,185.100.87.245,"{'185.100.87.245': [1578330728.879811, 1578330...","{'185.100.87.245': ['20200106_595_http', '2020...","[Romania, United States, Romania, Romania]","[Romania, United States]","{'Romania': 3, 'United States': 1}","{'Romania': 0.75, 'United States': 0.25}",Romania,"[http, smb, smtp]","[20200106_0_smb, 20200106_11116_smtp, 20200106...","{'20200106_1846_http': 373, '20200106_1700_htt...","{'20200106_1846_http': 0.2909516380655226, '20...",4,2,22,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,211.38.159.129,"{'211.38.159.129': [1578270324.174275, 1578796...","{'211.38.159.129': ['20200106_1610_ftp', '2020...","[South Korea, Thailand, China]","[China, South Korea, Thailand]","{'South Korea': 1, 'Thailand': 1, 'China': 1}","{'South Korea': 0.3333333333333333, 'Thailand'...",South Korea,[ftp],"[20200106_1610_ftp, 20200106_85899383096_ftp]","{'20200106_1610_ftp': 4, '20200106_85899383096...","{'20200106_1610_ftp': 0.5714285714285714, '202...",3,3,2,1
361,196.52.43.119,"{'196.52.43.119': [1578601951.741461, 15787056...","{'196.52.43.119': ['20200106_595_http', '20200...","[South Africa, Albania]","[Albania, South Africa]","{'South Africa': 1, 'Albania': 1}","{'South Africa': 0.5, 'Albania': 0.5}",South Africa,"[ftp, http]","[20200106_1586_ftp, 20200106_595_http]","{'20200106_595_http': 3, '20200106_1586_ftp': 3}","{'20200106_595_http': 0.5, '20200106_1586_ftp'...",2,2,2,2
362,45.143.222.191,"{'45.143.222.191': [1578239432.430883, 1578239...","{'45.143.222.191': ['20200106_622_smtp', '2020...","[Netherlands, Netherlands, China, China, China...","[China, Netherlands, Poland]","{'China': 3, 'Netherlands': 2, 'Poland': 1}","{'China': 0.5, 'Netherlands': 0.33333333333333...",China,[smtp],"[20200106_622_smtp, 20200106_8589962662_smtp]","{'20200106_622_smtp': 8252, '20200106_85899626...","{'20200106_622_smtp': 0.99457635289863

In [38]:
all_clusters = pattern_select_df['cluster_set'].tolist()
all_clusters = list(functools.reduce(operator.iconcat, all_clusters, []))
all_clusters = dict(Counter(all_clusters))
all_clusters = {k: v for k, v in sorted(all_clusters.items(), key=lambda item: item[1],reverse=True)}
all_clusters 

{'20200106_595_http': 146,
 '20200106_1700_http': 125,
 '20200106_2190_http': 66,
 '20200106_1440_http': 62,
 '20200107_6_ssh': 56,
 '20200106_13357_http': 55,
 '20200106_249108104923_http': 50,
 '20200106_0_smb': 44,
 '20200112_216_ssh': 41,
 '20200106_291_ssh': 38,
 '20200106_1846_http': 35,
 '20200106_1633_http': 33,
 '20200106_1586_ftp': 31,
 '20200106_1587_http': 30,
 '20200106_7465_http': 26,
 '20200106_23217_http': 24,
 '20200106_8355_pop': 20,
 '20200109_8_ssh': 16,
 '20200106_3315_http': 15,
 '20200106_3180_http': 14,
 '20200106_77309421989_http': 13,
 '20200106_1610_ftp': 12,
 '20200108_94_ssh': 12,
 '20200106_60129544415_http': 12,
 '20200106_39176_http': 11,
 '20200106_11929_smb': 11,
 '20200106_2131_ftp': 10,
 '20200106_22594_smtp': 10,
 '20200106_2877_http': 10,
 '20200106_68719508588_http': 10,
 '20200106_17179895470_pop': 9,
 '20200106_60129544476_ftp': 8,
 '20200106_622_smtp': 8,
 '20200107_8589984478_ftp': 8,
 '20200106_23675_http': 8,
 '20200106_3314_http': 8,
 '2020

In [39]:
pickle.dump(obj=all_clusters,file=open(pickle_dir+'clusterINgroup_stat_'+str(thr)+'_'+str(min_date)+'_'+str(max_date)+'.pkl','wb')) #改
print("手法(clusters)出現次數頻率:",pickle_dir+'clusterINgroup_stat_'+str(min_date)+'_'+str(max_date)+'.pkl')

手法(clusters)出現次數頻率: /home/antslab/NAS1_RAID6/pcap_inter/2020_01_06/台灣大哥大/case_pickles/clusterINgroup_stat_20200106_20200112.pkl


不同的IP會做哪些事情

In [40]:
%%time
def find_country(ip):
    return ip_df[ip_df['src_ip'] == ip]['session_county'].iloc[0]
jc_matrix3 = jc_matrix2.reset_index()
jc_matrix3['country'] = jc_matrix3['index'].map(find_country)


CPU times: user 16 s, sys: 334 ms, total: 16.3 s
Wall time: 16.3 s


In [41]:
jc_matrix_country = jc_matrix3.groupby('country').sum()
jc_matrix_country_final = jc_matrix3.groupby('country').sum()

In [42]:
jc_matrix_country = jc_matrix3.groupby('country').sum()
jc_matrix_country_final = jc_matrix3.groupby('country').sum()
jc_matrix_country_final['max_behavior'] = jc_matrix_country.idxmax(axis=1)
s = pd.Series(jc_matrix_country.idxmax(axis=0), name="max_country")
jc_matrix_country_final = jc_matrix_country_final.append(s)
pickle.dump(file=open(pickle_dir+'country_behavior_table_'+str(min_date)+'_'+str(max_date)+'.pkl','wb'),obj=jc_matrix_country_final)
print('Country Cluster df save in:',pickle_dir+'country_behavior_table_'+str(min_date)+'_'+str(max_date)+'.pkl')
jc_matrix_country_final


Country Cluster df save in: /home/antslab/NAS1_RAID6/pcap_inter/2020_01_06/台灣大哥大/case_pickles/country_behavior_table_20200106_20200112.pkl


,20200106_23217_http,20200106_595_http,20200106_1700_http,20200106_249108104923_http,20200106_1440_http,20200106_7465_http,20200106_1846_http,20200106_3315_http,20200106_23657_http,20200106_3180_http,...,20200106_48295_telnet,20200111_2230_telnet,20200110_12842_telnet,20200111_77309424108_telnet,20200111_42949694294_telnet,20200112_24491_telnet,20200106_13657_sip,20200106_1589_sip,20200106_26617_sip,max_behavior
country,,,,,,,,,,,,,,,,,,,,,
Afghanistan,0,3,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,20200106_595_http
Albania,0,23,23,0,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,20200106_595_http
Algeria,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,20200106_8589982593_http
Antigua and Barbuda,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,20200106_595_http
Argentina,0,41,45,0,5,0,4,1,0,0,...,0,0,0,0,0,0,0,0,0,20200107_6_ssh
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yemen,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,20200106_291_ssh
Zambia,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,20200106_291_ssh
Zimbabwe,0,6,3,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,20200106_595_http


In [ ]:
jc_matrix_country.nlargest(3,jc_matrix_country.columns)

將patern select df合併經緯度、抓出ssdeep hash

In [ ]:
#可不跑
test_path = pickle_dir+'CorrelateIP_DRAW_stat0.9.pkl' #改thr
paatern_select_df = pickle.load(open(test_path,'rb'))
paatern_select_df

In [ ]:
# Deprecated! Need Fix!!#Need Fix intention dict
def convert2intention(cluster_name):
    try:
        intention = intention_dict[cluster_name]
    except KeyError:
        intention = 'probing'
    return intention
all_df['intention'] = all_df.idx.apply(convert2intention)
all_df

In [43]:
%%time
import geoip2.database
from geoip2.errors import AddressNotFoundError
#改
city_reader = geoip2.database.Reader('/home/antslab/NAS1_RAID6/GeoIP2-DB/GeoIP2-City_20200526/GeoIP2-City.mmdb')
city_reader_response = dict()
def find_lalo_all(ip_li):
    ip_df = pd.DataFrame(ip_li,columns=['ip'])
    def find_lalo(ip):
        try:
            city_response = city_reader.city(ip)
            latitude = city_response.location.latitude
            longitude = city_response.location.longitude
            return (latitude,longitude)
        except (AddressNotFoundError,NameError):
            return ('None','None')
    ip_df['lalo'] = ip_df['ip'].apply(find_lalo)
    return ip_df['lalo'].tolist()
all_df['lalo'] = all_df.src_ip.map(find_lalo_all)
all_df

CPU times: user 23min 9s, sys: 1.36 s, total: 23min 11s
Wall time: 23min 11s


,idx,timestamp,country,src_ip,ssdeep,protocol,session_num,lalo
0,20200106_23217_http,"[1578239430.609722, 1578239432.907151, 1578239...","[United States, United States, United States, ...","[66.249.64.245, 66.249.65.104, 66.249.71.53, 6...",[24:tA4ucXoajPt1HysrEA4s2XDCTjeN1dqgrFxtCBtDhL...,http,172870,"[(37.751, -97.822), (37.751, -97.822), (37.751..."
1,20200106_595_http,"[1578239430.767826, 1578239432.515023, 1578239...","[Iran, China, Belarus, China, China, Philippin...","[2.183.95.130, 49.234.29.65, 178.122.68.106, 4...",[12:JkHXq/IsheY1awh5UP+/4D7eh2x7AGGH46ZY1Zu8m:...,http,169244,"[(35.698, 51.4115), (34.7725, 113.7266), (53.8..."
2,20200106_1700_http,"[1578239431.472793, 1578239434.482427, 1578239...","[Canada, Colombia, United States, Canada, Cana...","[172.105.4.227, 181.129.123.202, 216.244.66.24...",[12:rHXq6x7YNDIsqCjh8wHhFMsml6n61HCY3NkZfsv8m:...,http,93561,"[(43.6547, -79.3623), (3.5861, -76.5011), (47...."
3,20200106_249108104923_http,"[1578289690.645383, 1578302389.776345, 1578338...","[United States, China, China, Morocco, China, ...","[154.202.13.17, 111.230.23.22, 49.234.96.87, 1...",[12:7KS/cHnMQ7s95HfSBBBBBBBBBBBBBSjmHXq/IYDr+b...,http,57702,"[(37.751, -97.822), (34.7725, 113.7266), (34.7..."
6,20200106_1440_http,"[1578239428.028117, 1578239433.566138, 1578239...","[Russia, United States, Kazakhstan, Belarus, U...","[46.146.126.112, 66.249.71.108, 85.29.166.247,...",[24:tAzfGCU6BRCTS18YH+w18YHYR385Hltb17KAzfGCU6...,http,34045,"[(58.0033, 56.2478), (37.751, -97.822), (51.18..."
...,...,...,...,...,...,...,...,...
822,20200112_24491_telnet,"[1578789078.671959, 1578800410.996216]","[United States, United States]","[204.93.154.214, 204.93.154.211]","[3:ro53fg5T:ro2N, 3:ro53fg5T:ro2N, 3:ro53fg5T:...",telnet,2,"[(37.751, -97.822), (37.751, -97.822)]"
823,20200110_688_sip,"[1578591003.982135, 1578591003.997796, 1578591...","[France, France, France, France, France, Franc...","[5.135.73.105, 5.135.73.105, 5.135.73.105, 5.1...",[24:pII7mDETT+c715k35PUkTR3liRQSi3H5Ki6T+c0b3Z...,sip,22276,"[(48.8582, 2.3387), (48.8582, 2.3387), (48.858..."
824,20200106_13657_sip,"[1578314626.450293, 1578314626.453001, 1578314...","[France, France, France, France, France, Franc...","[5.135.73.105, 5.135.73.105, 5.135.73.105, 5.1...",[48:0aauuILzzp+7aue08oKY18Vje821BCkOPk/Ix:Lfus...,sip,2045,"[(48.8582, 2.3387), (48.8582, 2.3387), (48.858..."
825,20200106_1589_sip,"[1578240132.276525, 1578240205.46512, 15782402...","[United States, United States, United States, ...","[104.206.128.26, 104.206.128.14, 104.206.128.7...",[24:qgOf3vI7mDL5gizTKiqf3vX4v/DRt0UUgbYMfy1wUX...,sip,1312,"[(37.751, -97.822), (37.751, -97.822), (37.751..."


In [44]:
pickle.dump(file=open(pickle_dir+str(min_date)+'_'+str(max_date)+'_clusterID_time_country_ip_ssdeep.pkl','wb')
            ,obj=all_df)
print("Cluster資訊df path:",
      pickle_dir+str(min_date)+'_'+str(max_date)+'_clusterID_time_country_ip_ssdeep_lalo.pkl')

Cluster資訊df path: /home/antslab/NAS1_RAID6/pcap_inter/2020_01_06/台灣大哥大/case_pickles/20200106_20200112_clusterID_time_country_ip_ssdeep_lalo.pkl


In [ ]:
#deprecated
def need_col(gb):
    d = {}
    country_li = gb['country'].tolist() #抓出group by後的country欄位，並把所有值轉換為list
    lalo_li = gb['lalo'].tolist()
    time_li = gb['timestamp'].tolist()
    d['country'] = list(functools.reduce(operator.iconcat, country_li, [])) #合併所有list為一個list
    d['lalo'] = list(functools.reduce(operator.iconcat, lalo_li, []))
    d['timestamp'] = list(functools.reduce(operator.iconcat, time_li, []))
    return pd.Series(d,index=['country','lalo','timestamp'])
def sort_li(time_li, country_li):
    '''
    GOAL: sort by time (align with time's order)
    Return: list
    '''
    sort_country_li = [x for _,x in sorted(zip(time_li,country_li))]
    return sort_country_li
# Deprecated! Need Fix!!#Need Fix intention dict
draw_intention_df = all_df.groupby('intention').apply(need_col)
draw_intention_df['country'] = draw_intention_df.apply(lambda x:sort_li(x.timestamp,x.country),axis=1)
draw_intention_df['lalo'] = draw_intention_df.apply(lambda x:sort_li(x.timestamp,x.lalo),axis=1)
draw_intention_df['timestamp'] = draw_intention_df.timestamp.map(sorted)
pickle.dump(file=open(pickle_dir+str(min_date)+'_'+str(max_date)+'_intention_country_lalo_drawdf.pkl','wb'),obj=draw_intention_df)
print("全球視覺化地圖df path:",
      pickle_dir+str(min_date)+'_'+str(max_date)+'_intention_country_lalo_drawdf.pkl')
draw_intention_df
# intention_lalo_df = pd.DataFrame(all_df.groupby(['intention'])['country'].apply(list))
# def combine_li(liolis):
#     return list(functools.reduce(operator.iconcat, liolis, []))
# print(intention_lalo_df.columns)
# intention_lalo_df['country'] = intention_lalo_df.country.map(combine_li)
# intention_lalo_df['lalo'] = intention_lalo_df.lalo.map(combine_li)
# intention_lalo_df.loc['brute_force_login','country']

In [ ]:
len(draw_intention_df['timestamp'].iloc[1])

# THE END
第一名: 146.88.51.234	、 Thailand
第二名: 101.4.136.34、China
第三名:194.167.44.91(117)、France
第四名:151.253.165.70(111)、United Arab Emirates阿拉伯聯合大公國
第五名:46.235.53.26(102)、Russia

AS statistics

In [2]:
AS09 = pickle.load(open('/home/antslab/NAS1_RAID6/pcap_inter/2020_01_06/中華電信/case_pickles/CorrelateIP_DRAW_stat0.9.pkl','rb'))
AS09

,pattern_key,sessions_time_dict,cluster_id_dict,country_list,country_set,country_nums,country_portion,main_country,proto_set,cluster_set,cluster_nums,cluster_portion,ip_num,unique_country_num,unique_cluster_num,unique_protocols_num
0,185.100.87.191,"{'185.100.87.191': [1578827158.647363, 1578827...","{'185.100.87.191': ['20200106_213_http', '2020...","[Romania, United States, Romania, Romania, Rom...","[Romania, United States]","{'Romania': 6, 'United States': 2}","{'Romania': 0.75, 'United States': 0.25}",Romania,"[http, imap, pop, smb, smtp]","[20200106_103079219020_pop, 20200106_120259105...","{'20200106_252_http': 1462, '20200106_3_http':...","{'20200106_252_http': 0.26742271812694346, '20...",8,2,59,5
1,198.143.158.82,"{'198.143.158.82': [1578325916.274164, 1578333...","{'198.143.158.82': ['20200106_213_http', '2020...","[United States, Netherlands, United States, Ne...","[Netherlands, United States]","{'United States': 3, 'Netherlands': 3}","{'United States': 0.5, 'Netherlands': 0.5}",United States,"[ftp, http, imap, pop, smtp, ssh, telnet]","[20200106_111669172379_ftp, 20200106_14910_smt...","{'20200106_14910_smtp': 366, '20200106_252_htt...","{'20200106_14910_smtp': 0.10964649490713002, '...",6,2,44,7
2,61.178.160.156,"{'61.178.160.156': [1578381678.520108, 1578381...","{'61.178.160.156': ['20200106_1893_ftp', '2020...","[China, China]",[China],{'China': 2},{'China': 1.0},China,[ftp],"[20200106_14464_ftp, 20200106_146028902569_ftp...","{'20200106_1893_ftp': 667, '20200107_301_ftp':...","{'20200106_1893_ftp': 0.6052631578947368, '202...",2,1,13,1
3,83.97.20.46,"{'83.97.20.46': [1578239450.921439, 1578239501...","{'83.97.20.46': ['20200106_15768_ftp', '202001...","[Romania, Romania]",[Romania],{'Romania': 2},{'Romania': 1.0},Romania,"[ftp, pop, sip, smb, smtp, telnet]","[20200106_15765_ftp, 20200106_15766_pop, 20200...","{'20200106_15768_ftp': 885, '20200106_15765_ft...","{'20200106_15768_ftp': 0.11920797413793104, '2...",2,1,12,6
4,89.19.29.189,"{'89.19.29.189': [1578671515.239016, 157867151...","{'89.19.29.189': ['20200106_252_http', '202001...","[Turkey, United States, United States]","[Turkey, United States]","{'United States': 2, 'Turkey': 1}","{'United States': 0.6666666666666666, 'Turkey'...",United States,[http],"[20200106_103079220028_http, 20200106_252_http...","{'20200106_3026_http': 1310, '20200106_417_htt...","{'20200106_3026_http': 0.5256821829855538, '20...",3,2,11,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
444,222.114.92.2,"{'222.114.92.2': [1578596966.708644, 157852673...","{'222.114.92.2': ['20200106_467_http', '202001...","[South Korea, Taiwan]","[South Korea, Taiwan]","{'South Korea': 1, 'Taiwan': 1}","{'South Korea': 0.5, 'Taiwan': 0.5}",South Korea,[http],"[20200106_467_http, 20200106_550_http]","{'20200106_550_http': 4, '20200106_467_http': 2}","{'20200106_550_http': 0.6666666666666666, '202...",2,2,2,1
445,208.100.26.233,"{'208.100.26.233': [1578593864.873624, 1578593...","{'208.100.26.233': ['20200106_14910_smtp', '20...","[United States, United States, United States, ...",[United States],{'United States': 4},{'United States': 1.0},United States,[smtp],"[20200106_14910_smtp, 20200110_8589939720_smtp]","{'20200110_8589939720_smtp': 184, '20200106_14...",{'20200110_8589939720_smtp': 0.534883720930232...,4,1,2,1
446,182.72.139.6,"{'182.72.139.6': [1578719750.797057, 157824108...","{'182.72.139.6': ['20200106_213_http', '202001...","[India, Russia]","[India, Russia]","{'India': 1, 'Russia': 1}","{'India': 0.5, 'Russia': 0.5}",India,"[http, ssh]","[20200106_213_http, 20200106_7_ssh]","{'20200106_7_ssh': 125, '20200106_213_http': 2}","{'20200106_7_ssh': 0.984251968503937, '2020010...",2,2,2,2
447,184.75.221.163,"{'184.75.221.163': [1578672169.451922, 1578673...","{'184.75.221.163': ['20200106_475_ftp', '20200...","[Canada, Germany]","[Canada, Germany]","{'Canada': 1, 'Germany': 1}","{'Canada': 0.5, 'Germany': 0.5}",Canada,[ftp],"[20200106_475_ftp, 20200106_8589950294_ftp]","{

In [8]:
sorted(AS09['ip_num'].tolist(),reverse=True)

[3685,
 1000,
 584,
 415,
 385,
 294,
 269,
 245,
 163,
 132,
 130,
 123,
 121,
 114,
 100,
 98,
 97,
 90,
 89,
 86,
 86,
 85,
 83,
 81,
 76,
 72,
 69,
 67,
 66,
 65,
 62,
 60,
 54,
 53,
 53,
 49,
 48,
 46,
 45,
 45,
 45,
 45,
 38,
 37,
 37,
 36,
 32,
 31,
 31,
 29,
 29,
 28,
 27,
 26,
 24,
 24,
 24,
 23,
 21,
 21,
 21,
 20,
 19,
 19,
 19,
 19,
 18,
 18,
 18,
 17,
 17,
 17,
 17,
 16,
 16,
 16,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 14,
 14,
 14,
 14,
 14,
 13,
 13,
 13,
 13,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 11,
 11,
 11,
 11,
 11,
 11,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,


In [3]:
tops = AS09[AS09['ip_num']>500]
tops

,pattern_key,sessions_time_dict,cluster_id_dict,country_list,country_set,country_nums,country_portion,main_country,proto_set,cluster_set,cluster_nums,cluster_portion,ip_num,unique_country_num,unique_cluster_num,unique_protocols_num
308,37.228.67.205,"{'37.228.67.205': [1578811817.03101, 157851092...","{'37.228.67.205': ['20200106_213_http', '20200...","[Kazakhstan, Ukraine, Iran, Russia, Iran, Russ...","[Afghanistan, Albania, Argentina, Armenia, Aus...","{'Brazil': 689, 'Russia': 333, 'India': 280, '...","{'Brazil': 0.1869742198100407, 'Russia': 0.090...",Brazil,[http],"[20200106_213_http, 20200106_252_http]","{'20200106_252_http': 9058, '20200106_213_http...","{'20200106_252_http': 0.6068198566356268, '202...",3685,119,2,1
310,5.154.40.6,"{'5.154.40.6': [1578362582.870325, 1578380939....","{'5.154.40.6': ['20200106_3_http', '20200106_3...","[Spain, United States, United States, Romania,...","[Argentina, Australia, Austria, Bahamas, Bangl...","{'United States': 173, 'Mexico': 109, 'Malaysi...","{'United States': 0.173, 'Mexico': 0.109, 'Mal...",United States,[http],"[20200106_14533_http, 20200106_3_http]","{'20200106_3_http': 4313, '20200106_14533_http...","{'20200106_3_http': 0.5683225721438925, '20200...",1000,75,2,1
312,37.192.16.123,"{'37.192.16.123': [1578444165.594689, 15786281...","{'37.192.16.123': ['20200106_252_http', '20200...","[Russia, Spain, Russia, Ukraine, Iraq, Czechia...","[Albania, Argentina, Australia, Austria, Bangl...","{'China': 125, 'Brazil': 73, 'Russia': 49, 'In...","{'China': 0.21404109589041095, 'Brazil': 0.125...",China,[http],"[20200106_252_http, 20200106_3026_http]","{'20200106_252_http': 1826, '20200106_3026_htt...","{'20200106_252_http': 0.6708302718589273, '202...",584,73,2,1


In [4]:
pickle.dump(obj=tops,file=open('/home/antslab/NAS1_RAID6/pcap_inter/2020_01_06/中華電信/case_pickles/AttackScenarioVIZ_DataDemo.pkl','wb'))

In [20]:
tops.loc[308,'country_portion']
# tops.loc[312,'cluster_portion']

{'Brazil': 0.1869742198100407,
 'Russia': 0.0903663500678426,
 'India': 0.07598371777476255,
 'Iran': 0.0485753052917232,
 'Indonesia': 0.04748982360922659,
 'Ukraine': 0.042333785617367704,
 'Turkey': 0.04151967435549525,
 'United States': 0.03419267299864315,
 'Poland': 0.030664857530529173,
 'Colombia': 0.021709633649932156,
 'Bangladesh': 0.021438263229308006,
 'Serbia': 0.01953867028493894,
 'Spain': 0.018995929443690638,
 'Czechia': 0.01818181818181818,
 'Italy': 0.015468113975576661,
 'Argentina': 0.014654002713704206,
 'Cambodia': 0.01383989145183175,
 'South Africa': 0.01383989145183175,
 'Greece': 0.013025780189959294,
 'Mexico': 0.011940298507462687,
 'Ecuador': 0.011668928086838535,
 'Bulgaria': 0.011397557666214383,
 'Kenya': 0.010040705563093623,
 'China': 0.008955223880597015,
 'Romania': 0.007327001356852103,
 'Albania': 0.0067842605156037995,
 'Thailand': 0.006512890094979647,
 'Australia': 0.006512890094979647,
 'Chile': 0.006241519674355495,
 'Honduras': 0.0059701492

In [4]:
AS_loner = pickle.load(open('/home/antslab/NAS1_RAID6/pcap_inter/2020_01_06/中華電信/case_pickles/loner_draw_country0.9.pkl','rb'))
AS_loner

,src_ip,session_timelist,session_idlist,session_county
0,185.100.87.248,"[1578333491.029024, 1578800825.165391, 1578827...","[20200106_213_http, 20200106_213_http, 2020010...",Romania
1,208.100.26.229,"[1578827062.502704, 1578329385.835335, 1578329...","[20200106_213_http, 20200106_252_http, 2020010...",United States
2,185.100.87.246,"[1578329839.202812, 1578329841.130075, 1578329...","[20200106_252_http, 20200106_252_http, 2020010...",Romania
3,211.95.50.5,"[1578733476.160312, 1578733482.053956, 1578733...","[20200106_213_http, 20200106_213_http, 2020010...",China
4,211.95.50.4,"[1578733367.027604, 1578733380.936623, 1578733...","[20200106_213_http, 20200106_213_http, 2020010...",China
...,...,...,...,...
686,185.195.237.25,"[1578253340.0843, 1578349232.978212, 157861148...","[20200106_482_http, 20200106_7_ssh, 20200106_7...",None
687,178.242.23.197,"[1578388854.848924, 1578389599.807857, 1578389...","[20200106_4080_http, 20200106_120259116547_tel...",Turkey
688,18.191.238.10,"[1578754909.274516, 1578755414.239824, 1578757...","[20200106_68719507596_ftp, 20200106_6871950759...",United States
689,185.65.134.176,"[1578421721.904243, 1578428494.247924, 1578431...","[20200106_14464_ftp, 20200106_14464_ftp, 20200...",Netherlands


輸出clique圖的case study (跨天、跨protocol)
* Deprecated

In [ ]:
#輸出clique圖的case study (跨天、跨protocol)
clique_dir = '/home/antslab/spark_data/pcap_inter/2020_01_06/中華電信/case_pickles/clique/'
save_path_li = []
file_names_li = next(os.walk(clique_dir))[2]
file_names_li = list(filter(lambda k: 'clique_' in k, file_names_li))
file_names_li = list(filter(lambda k: 'cluster' not in k, file_names_li))
file_names_li = list(filter(lambda k: 'http' not in k, file_names_li))
len(file_names_li)
file_names_li

In [ ]:
def convert_date(s):
    '''
    將日期加上月份
    '''
    mo = '01' #改?
    date = str(s)
    if len(date)==1:
        return mo+'0'+date
    elif len(date)==2:
        return mo+date
    elif len(date)==4:
        return date

In [ ]:
#輸出clique圖的case study (跨天、跨protocol)
for file_name in tqdm(file_names_li):
# in_file = '/home/antslab/spark_data/pcap_inter/2020_01_06/中華電信/case_pickles/clique_34459.pkl'
    in_file = clique_dir+file_name
    out_filename = in_file.split('/')[-1].split('.')[0]
#     if out_filename != 'clique_34478': #DEBUG!!
#         continue
    list_tuples = pickle.load(open(in_file,'rb'))
    case_wireshark_df = pd.DataFrame(list_tuples,columns =['Date', 'Protocol', 'SessionTime'])
    case_wireshark_df['Date'] = case_wireshark_df.Date.map(convert_date)
    case_wireshark_df = case_wireshark_df.sort_values(['Date','Protocol'],ascending=[True,True])
    case_wireshark_df = case_wireshark_df.reset_index(drop=True)
    
    date_li = list(set(case_wireshark_df.Date.tolist()))
    date_li = sorted(date_li)
    for i,date in enumerate(date_li):
#         print('===',date,'===') #DEBUG!!!
        date_df = case_wireshark_df[case_wireshark_df.Date == date]
        all_time_li = [] #所有protocl一起存放成一個當案
        proto_li = list(set(date_df.Protocol.tolist()))
        proto_li = sorted(proto_li)
        for proto in proto_li:
#             if proto == 'imap' or proto == 'pop': ##DEBUG
#                 print(proto,date)
            pickle_dir = '/home/antslab/spark_data/pcap_inter/2020_'+str(date[:2])+'_'+str(date[-2:])+'/中華電信/case_pickles/'
            try:
                (proto_df, proto_df_payload,proto_big_dict,proto_loners,proto_score,proto_cluster_score_dict,
                                        proto_upgma_dict,stat_df) = pickle.load(open(pickle_dir+str(date)+'_'+str(proto)+'_all.pkl','rb'))
            except ValueError:
                (proto_df, proto_df_payload,proto_big_dict,proto_loners,proto_score,proto_cluster_score_dict,
                                        proto_upgma_dict) = pickle.load(open(pickle_dir+str(date)+'_'+str(proto)+'_all.pkl','rb'))
            except FileNotFoundError:
#                 print('!!NO FILE:',date,proto,'!!')
                date_tmp = str(int(date)+1)
                if len(date_tmp)==3: #限用同月份
                    date_tmp = '0'+date_tmp
                pickle_dir = '/home/antslab/spark_data/pcap_inter/2020_'+str(date_tmp[:2])+'_'+str(date_tmp[-2:])+'/中華電信/case_pickles/'
                try:
                    (proto_df, proto_df_payload,proto_big_dict,proto_loners,proto_score,proto_cluster_score_dict,
                                            proto_upgma_dict,stat_df) = pickle.load(open(pickle_dir+str(date_tmp)+'_'+str(proto)+'_all.pkl','rb'))
                except ValueError:
                    (proto_df, proto_df_payload,proto_big_dict,proto_loners,proto_score,proto_cluster_score_dict,
                                            proto_upgma_dict) = pickle.load(open(pickle_dir+str(date_tmp)+'_'+str(proto)+'_all.pkl','rb'))
                except FileNotFoundError:
                    print('!!NO FILE:',date_tmp,proto,'!!')
                    continue                

            date_proto_df = date_df[date_df.Protocol == proto]
            session_time_li = date_proto_df.SessionTime.tolist()
            need_df = proto_df[proto_df.session_time.isin(session_time_li)]
            not_today_time = list(set(session_time_li) - set(need_df.session_time.tolist()))
            try:
                time_lists = need_df.session_time_list.tolist()
            except AttributeError:
                time_lists = need_df.session_time_list.values
            time_lists = [list(x) for x in time_lists]
            all_time_li.extend(time_lists)
            if len(not_today_time)>1: #有非今天的session time
                date_tmp = str(int(date)+1)
                if len(date_tmp)==3:
                    date_tmp = '0'+date_tmp
                date_df = case_wireshark_df[case_wireshark_df.Date == date_tmp]
                pickle_dir = '/home/antslab/spark_data/pcap_inter/2020_'+str(date_tmp[:2])+'_'+str(date_tmp[-2:])+'/中華電信/case_pickles/'
                try:
                    (proto_df, proto_df_payload,proto_big_dict,proto_loners,proto_score,proto_cluster_score_dict,
                                            proto_upgma_dict,stat_df) = pickle.load(open(pickle_dir+str(date_tmp)+'_'+str(proto)+'_all.pkl','rb'))
                except ValueError:
                    (proto_df, proto_df_payload,proto_big_dict,proto_loners,proto_score,proto_cluster_score_dict,
                                            proto_upgma_dict) = pickle.load(open(pickle_dir+str(date_tmp)+'_'+str(proto)+'_all.pkl','rb'))
                except FileNotFoundError:
                    print('!!NO FILE:',date_tmp,proto,'!!')
                    continue
                date_proto_df = date_df[date_df.Protocol == proto]
                session_time_li = date_proto_df.SessionTime.tolist()
                need_df = proto_df[proto_df.session_time.isin(session_time_li)]
                try:
                    time_lists = need_df.session_time_list.tolist()
                except AttributeError:
                    time_lists = need_df.session_time_list.values
                time_lists = [list(x) for x in time_lists]
                if len(time_lists)>1:
                    all_time_li.extend(time_lists)                
                
                
        out_path = pickle_dir+out_filename+'_session_time_list.pkl'
        pickle.dump(file=open(out_path,'wb'),obj=all_time_li)
        save_path_li.append(out_path)
#         if 'clique_34478' in out_path: #DEBUG
#             print(out_filename,date,':',out_path)
pickle.dump(obj=save_path_li,file=open(clique_dir+'ALL_session_time_lists.pkl','wb')) #debug
print('Meta Path Save in :',clique_dir+'ALL_session_time_lists.pkl')        
#         print(out_filename,date,':',out_path)
# print(out_filename)
# case_wireshark_df

In [ ]:
# date_li = list(set(case_wireshark_df.Date.tolist()))
# date_li = sorted(date_li)
# for date in date_li:
#     date_df = case_wireshark_df[case_wireshark_df.Date == date]
#     all_time_li = [] #所有protocl一起存放成一個當案
#     proto_li = list(set(date_df.Protocol.tolist()))
#     proto_li = sorted(proto_li)
#     for proto in proto_li:
#         pickle_dir = '/home/antslab/spark_data/pcap_inter/2020_'+str(date[:2])+'_'+str(date[-2:])+'/中華電信/case_pickles/'
#         try:
#             (proto_df, proto_df_payload,proto_big_dict,proto_loners,proto_score,proto_cluster_score_dict,
#                                     proto_upgma_dict,stat_df) = pickle.load(open(pickle_dir+str(date)+'_'+str(proto)+'_all.pkl','rb'))
#         except ValueError:
#             (proto_df, proto_df_payload,proto_big_dict,proto_loners,proto_score,proto_cluster_score_dict,
#                                     proto_upgma_dict) = pickle.load(open(pickle_dir+str(date)+'_'+str(proto)+'_all.pkl','rb'))
#         except FileNotFoundError:
#             print('!!NO FILE:',date,proto,'!!')
#             continue
#         date_proto_df = date_df[date_df.Protocol == proto]
#         session_time_li = date_proto_df.SessionTime.tolist()
#         need_df = proto_df[proto_df.session_time.isin(session_time_li)]
#         try:
#             time_lists = need_df.session_time_list.tolist()
#         except AttributeError:
#             time_lists = need_df.session_time_list.values
#         time_lists = [list(x) for x in time_lists]
#         all_time_li.extend(time_lists)
#     out_path = pickle_dir+out_filename+'_session_time_list.pkl'
#     pickle.dump(file=open(out_path,'wb'),obj=all_time_li)
#     print(out_filename,date,':',out_path)

         

In [ ]:
# #可以直接畫好trend圖、backup plan
# similarity_id_list = []
# timelist_dict_list = []
# clusterlist_dict_list = []
# jc_matrix2 = jc_matrix.drop(['np_array'],axis=1)
# for cluster_id, ip_li in tqdm(jc_dict.items()):
#     similarity_id_list.append(cluster_id) #僅識別用
#     all_ips = ip_li[:]
#     all_ips.append(cluster_id)
#     timelist_dict = {}
#     clusterlist_dict = {}
#     for t_ip in all_ips: #這個IP有橫跨的IDX，都屬於這個ip的
#         temp = jc_matrix2[jc_matrix2.index == t_ip]
#         t_idx = temp.columns[temp.eq(1).any()]
#         temp2 = all_df[all_df.idx.isin(t_idx)]
#         temp2['gen'] = temp2.src_ip.apply(find,args=(t_ip,))
#         temp2['time_li'] = temp2.apply(lambda x: find_time(x.gen, x.timestamp), axis=1)
#         temp2['idx_li'] = temp2.apply(lambda x: repeat_idx(x.time_li, x.idx), axis=1)        
#         timestamps_lists = temp2['time_li'].tolist() #list of lists
#         cluster_name_list = temp2['idx_li'].tolist()#list of lists
#         timelist_dict[t_ip] = functools.reduce(operator.iconcat, timestamps_lists, [])
#         clusterlist_dict[t_ip] = functools.reduce(operator.iconcat, cluster_name_list, [])
        
#     timelist_dict_list.append(timelist_dict)
#     clusterlist_dict_list.append(clusterlist_dict)
#     assert len(similarity_id_list) == len(timelist_dict_list) == len(clusterlist_dict_list)
# pattern_df = pd.DataFrame([similarity_id_list,timelist_dict_list,clusterlist_dict_list],
#              index=['pattern_key','sessions_time_dict','cluster_id_dict']).T
# pattern_df

In [ ]:
# def find(myList,target ):
#     return [i for i,j in enumerate(myList) if j == target]
# #     for i in range(len(myList)):
# #         if myList[i] == target:
# #             yield i
# def find_time(indexes,li):
# #     print(*idx_li)
# #     print(li)
# #     idx_li = list(gen)
#     return list(map(li.__getitem__, indexes))
# #     return list(itemgetter([*idx_li])(li))
# def repeat_idx(ori_li,index):
#     return [index]*len(ori_li)

In [ ]:
# def find(myList,target ):
#     for i in range(len(myList)):
#         if myList[i] == target:
#             yield i
# l = ['aa','bb','aa']
# kk = list(find('aa',l)) #index
# # for k in kk:
# #     print(k)
# import operator
# list(itemgetter(*kk)(l)) #element
# kk

In [ ]:
#可以畫trend圖
def ip_isin(li,t_ip):
    '''
    input: src_ip list & target ip
    Return True False
    '''
    np_array = np.array(li)
    item_index = np.where(np_array==t_ip)
    item_index = list(item_index[0])
    if len(item_index)>0:
        return item_index
    else:
        return False
    
def get_timestamps(need_li,idx_li):
    '''
    GOAL: use idx list to get time stamps
    '''
    need_np = np.array(need_li)
    return list(need_np[idx_li])

def get_country(need_li,idx_li):
    '''
    GOAL: use idx list to get country
    '''
    return need_li[idx_li[0]]

In [ ]:
#可以畫trend圖
similarity_id_list = []
timelist_dict_list = []
clusterlist_dict_list = []
all_df2 = all_df[:]
for cluster_id, ip_li in tqdm(jc_dict.items()):
    similarity_id_list.append(cluster_id) #僅識別用
    all_ips = ip_li[:]
    all_ips.append(cluster_id)
    timelist_dict = {}
    clusterlist_dict = {}
    for t_ip in all_ips: #這個IP有橫跨的IDX，都屬於這個ip的
        all_df2['need_idx'] = all_df2.src_ip.apply(ip_isin,args=(t_ip,))
        all_df3 = all_df2[all_df2['need_idx']!=False]
        all_df3['need_timestamp'] = all_df3.apply(lambda x: get_timestamps(x.timestamp, x.need_idx), axis=1)
#         all_df3['need_country'] = all_df3.apply(lambda x: get_country(x.country, x.need_idx), axis=1)
        timestamps_lists = all_df3['need_timestamp'].tolist()
        cluster_name_list = all_df3['idx'].tolist()
        assert len(timestamps_lists) == len(cluster_name_list)
        cluster_names_li = []
        for i,timestamp_li in enumerate(timestamps_lists):
            for _ in range(len(timestamp_li)):
                cluster_names_li.append(cluster_name_list[i])
        final_ip_timestamps = sum(timestamps_lists,[])
        assert len(cluster_names_li) == len(final_ip_timestamps)
        clusterlist_dict[t_ip] = cluster_names_li
        timelist_dict[t_ip] = final_ip_timestamps
    timelist_dict_list.append(timelist_dict)
    clusterlist_dict_list.append(clusterlist_dict)
    assert len(similarity_id_list) == len(timelist_dict_list) == len(clusterlist_dict_list)
pattern_df = pd.DataFrame([similarity_id_list,timelist_dict_list,clusterlist_dict_list],
             columns=['pattern_id','sessions_time_dict','cluster_id_dict'])
pattern_df

In [ ]:
# for ip in tqdm(all_ips):
#     def find_cluster_idx(src_ip_li):
#         '''
#         GOAL: get cluster_id
#         '''
#         if ip in src_ip_li:
#             return True
#         else:
#             return False
#     all_df['temp_in'] = all_df.src_ip.map(find_cluster_idx)
#     c_id_df = all_df[all_df.temp_in == True]
#     c_id_df = c_id_df.idx.tolist()
#     jc_matrix.loc[ip,c_id_df] = 1
# jc_matrix

In [ ]:
def count_distinct_number(li):
    return len(set(li))

In [ ]:
http_df['ip_num'] = http_df.src_ip.map(count_distinct_number)
http_df['ip_num']

小群關聯 (Deprecated)
* L、S
* src_ip

In [ ]:
loner_protocol_dict = {}
loner_time_dict = {}
date_li = ["0110","0111","0109","0112","0108","0107","0106"] #,"0102","0130"
proto_li = ['http','smb','telnet','ftp','smtp','mysql','ssh','rpc','imap','pop','sip']
# for proto in tqdm(proto_li):
for date in date_li:
    pickle_dir = '/home/antslab/spark_data/pcap_inter/2020_01_'+str(date[-2:])+'/中華電信/case_pickles/'
    try:
        (proto_df, proto_df_payload,proto_big_dict,proto_loners,proto_score,proto_cluster_score_dict,
                                proto_upgma_dict,stat_df) = pickle.load(open(pickle_dir+str(date)+'_'+str(proto)+'_all.pkl','rb'))
    except ValueError:
        (proto_df, proto_df_payload,proto_big_dict,proto_loners,proto_score,proto_cluster_score_dict,
                                proto_upgma_dict) = pickle.load(open(pickle_dir+str(date)+'_'+str(proto)+'_all.pkl','rb'))
    except FileNotFoundError:
        continue
    for k,v in proto_upgma_dict.items():
        if '_L_' not in k: #只拿Loner來做關聯
            continue
        lonerInfo = k.split('_')
        loner_ip = lonerInfo[-1]
        loner_counry = lonerInfo[-3]
        loner_domain = lonerInfo[-2]
        for d in date_li:
            in_file = '/home/antslab/data_hdd4t/pcap_process/pcap_spark/2020_01_'+str(d[-2:])+'/中華電信/parquet/2020_01_'+str(date[-2:])+'_session_v7-2_hashpayload_time_size.parquet' #改!!
            df_loner = ks.read_parquet(in_file)
            ip_df = df_loner[df_loner.ip_src == loner_ip]
            ip_df = ip_df.to_pandas()
            try:
                ori_loner_time_li = loner_time_dict['L_'+loner_counry+'_'+loner_domain+'_'+loner_ip][:]
                ori_loner_protocol_li = loner_protocol_dict['L_'+loner_counry+'_'+loner_domain+'_'+loner_ip][:]
                ori_loner_time_li.extend(ip_df.session_time.tolist())
                ori_loner_protocol_li.extend(ip_df.frame_i_max_protocols.tolist())
                loner_time_dict['L_'+loner_counry+'_'+loner_domain+'_'+loner_ip] = ori_loner_time_li
                loner_protocol_dict['L_'+loner_counry+'_'+loner_domain+'_'+loner_ip] = ori_loner_protocol_li
            except KeyError:
                loner_time_dict['L_'+loner_counry+'_'+loner_domain+'_'+loner_ip] = ip_df.session_time.tolist()
                loner_protocol_dict['L_'+loner_counry+'_'+loner_domain+'_'+loner_ip] = ip_df.frame_i_max_protocols.tolist()
    break #debug
#     break #debug

In [ ]:
proto_upgma_dict

In [ ]:
proto_big_dict

In [ ]:
proto_df_payload

#### Deprecated
畫distribution
* 針對某單一case進行

In [ ]:
def case_study(idx,payload_df):
    '''
    idx: which index is the dataframe's target?
    payload_df: protocol payload dataframe (packet-based)
    下面有一些參數可以改 (line19~23、27註解處)
    
    Return: Big dict
    '''
    big_dict = {}
    idx = int(idx)
    target = payload_df[payload_df.idx == idx]
    target = target.reset_index(drop=True)
    t_len = len(target)
    t_q1 = target.iloc[int(t_len*0.25),-1]
    t_q2 = target.iloc[int(t_len*0.5),-1]
    t_q3 = target.iloc[int(t_len*0.75),-1]
    candidate = payload_df[payload_df.idx != idx]
    candidate['idx'] = candidate.idx.astype(str)
    candidate_q1 = candidate[candidate['size']==target.iloc[int(t_len*0.25),-2]] #是否要match: match改成== 不match改成!=
    candidate['q1'] = ssdeep_compare(t_q1,candidate_q1)
    candidate_q2 = candidate[candidate['size']==target.iloc[int(t_len*0.5),-2]] # == or != (score=0)
    candidate['q2'] = ssdeep_compare(t_q2,candidate_q2)
    candidate_q3 = candidate[candidate['size']==target.iloc[int(t_len*0.75),-2]] #要找不像的時候要改成!=
    candidate['q3'] = ssdeep_compare(t_q3,candidate_q3)
    candidate = candidate.drop(['size','hash'],axis=1)
    candidate['score'] = candidate.max(axis=1)
    candidate = candidate[candidate.score>0] #相似度分數，數字越小速度愈快、數字越大越多群。要抓出幾分的人> == <
    candidate['idx'] = candidate.idx.astype(int)
    idx_li = candidate.idx.tolist()
    if len(idx_li)>0:
        big_dict[idx] = idx_li
    # len(big_dict.keys())
    print("所有該protocol與所給定之target的分數種類:",candidate.score.unique())
    print("總session數量:",len(big_dict[idx]))
    print("分數的分布:",candidate.score.value_counts().sort_index())
    print("分數的分布圖:",candidate.score.hist(bins=25))
    return big_dict, candidate

In [ ]:
candidate.score.value_counts().sort_index()#.sum()

In [ ]:
print(candidate.score.hist(bins=25))

In [ ]:
candidate.score.describe()

In [ ]:
def save_wireshark(df2_protocol,pkl_name,only_one=False):
    '''
    GOAL: save frame time list to pkl for Jerry to output wireshark pcap
    df2_protocol: protocol original dataframe (session-based)
    '''
    protocol_thr = df2_protocol.loc[big_dict_sqlmap[idx]]
    # protocol_thr = df2_protocol.loc[80891]  #也可只輸出一個特定sessiono的time frame list就好
    protocol_thr = protocol_thr.session_time_list.tolist()
    if not only_one:
        protocol_thr = [list(x) for x in protocol_thr]
    pickle.dump(obj=protocol_thr,file=open(pickle_dir+str(pkl_name)+'.pkl','wb'))
    print("總共具有的sessions數量:",len(protocol_thr))
# protocol_thr

sql map http

In [ ]:
df2_http[df2_http.ip_src == '190.92.5.158'].index

In [ ]:
for k,v in big_dict_http.items():
    if 80891 in v:
        print(k)
        break

In [ ]:
case1_diagram = df2_http.loc[list(set(big_dict_http[2394]))]
time_lists = case1_diagram.session_time_list.tolist()
time_lists = [list(x) for x in time_lists]
ips = case1_diagram.ip_src.tolist()
countries = case1_diagram.country.tolist()
print(len(countries),len(ips))
# pickle_dir = '/home/antslab/spark_data/pcap_inter/2020_01_10/中華電信/case_pickles/'
pickle.dump(obj=(time_lists,ips,countries),file=open(pickle_dir+'case#1_threetuples.pkl','wb'))

In [ ]:
case1_diagram.ip_src.unique()

TEST
* Decode char()

In [ ]:
temp[0][0]

In [ ]:
kk = 'char(91)+char(88)+char(93)+char(91)+char(88)+char(88)+char(93)'
kk = kk.replace('char(','')
kk = kk.replace(')','')
kk_li = kk.split('+')
words = []
for k in kk_li:
    if '(' in k:
        words.append('(')
        words.append(chr(int(k[1:])))
#         print('(',chr(int(k[1:])))
    elif ')' in k:
        words.append(chr(k[:-1]))
        words.append(')')
#         print(chr(k[:-1]),')')
    else:
        words.append(chr(int(k)))
#         print(chr(int(k)))
# chr(kk)
"".join(words)

In [ ]:
'CHAR(45,120,49,45,81,45)'
print(chr(45),chr(120),chr(49),chr(45),chr(81),chr(45))

For statistics

In [3]:
df_http_0110 = pickle.load(open('/home/antslab/NAS1_RAID6/pcap_inter/2020_01_10/中華電信/case_pickles/20200110_http_all.pkl','rb'))
df_http_0110

KeyboardInterrupt: 

In [4]:
path = '/home/antslab/NAS1_RAID6/pcap_inter/2020_01_10/中華電信/case_pickles/20200110_http_all.pkl'
try:
    (proto_df, proto_df_payload,proto_big_dict,proto_loners,proto_score,proto_cluster_score_dict,
     proto_upgma_dict,stat_df) = pickle.load(open(path,'rb'))
except ValueError:
    try:
        (proto_df, proto_df_payload,proto_big_dict,proto_loners,proto_score,proto_cluster_score_dict,
         proto_upgma_dict,stat_df,df2) = pickle.load(open(path,'rb'))
    except ValueError:
        try:
            (proto_df, proto_df_payload,proto_big_dict,proto_loners,proto_score,proto_cluster_score_dict,
                                    proto_upgma_dict,stat_df) = pickle.load(open(path,'rb'))
        except ValueError:
            (proto_df, proto_df_payload,proto_big_dict,proto_loners,proto_score,proto_cluster_score_dict,
                                    proto_upgma_dict) = pickle.load(open(path,'rb'))
proto_df

,session_time,session_time_list,session_duration,session_i_tt_packet,session_o_tt_packet,session_i_tt_frame_length,session_o_tt_frame_length,udp_i_tt_length,udp_o_tt_length,udp_i_avg_length,...,ip_dst,tcp_srcport,tcp_dstport,country,isp,domain,frame_i_max_protocols,frame_o_max_protocols,tcp_i_payload_list,tcp_o_payload_list
2,1.578668e+09,"[1578668067.012655, 1578668067.012826, 1578668...",0.118949,5,5,418,384,NaN,NaN,NaN,...,210.242.149.106,4038,80,Taiwan,None,hinet.net,eth:ethertype:ip:tcp:http,eth:ethertype:ip:tcp:http,[[12:qYllIOHXqLHHUA+5UZFulExmMReT2kC+m:qgllajH...,"[[3:KD+U/kakXbVfLu:KKAkakXU, 1578668067.110543..."
3,1.578668e+09,"[1578668053.538169, 1578668053.538542, 1578668...",0.285563,5,5,417,384,NaN,NaN,NaN,...,210.242.149.71,14968,80,Taiwan,None,hinet.net,eth:ethertype:ip:tcp:http,eth:ethertype:ip:tcp:http,[[6:uumYlloZOHKAqEiTBHE7T42pAy6A5gUA7i+stfXpUe...,"[[3:KD+U/kakXbVfLu:KKAkakXU, 1578668053.742663..."
110,1.578586e+09,"[1578586194.253401, 1578586194.253669, 1578586...",7.824246,6,7,422,566,NaN,NaN,NaN,...,210.242.145.73,3465,80,Vietnam,None,None,eth:ethertype:ip:tcp:http,eth:ethertype:ip:tcp:http:data,"[[3:cRm5tD+U/gfun:rHKAgmn, 1578586194.35948200...","[[3:KD+U/kakXbVfLu:KKAkakXU, 1578586201.247124..."
114,1.578606e+09,"[1578606396.579509, 1578606396.579728, 1578606...",8.179993,6,6,422,406,NaN,NaN,NaN,...,210.242.148.40,38419,8080,United States,None,verizon.net,eth:ethertype:ip:tcp:http,eth:ethertype:ip:tcp:http:data,"[[3:cRm5tD+U/2:rHKA2, 1578606396.777954000, 14...","[[3:Sn:Sn, 1578606404.562886000, 2]]"
117,1.578592e+09,"[1578591683.621066, 1578591683.621245, 1578591...",0.533666,5,5,516,740,NaN,NaN,NaN,...,210.242.146.81,11131,80,Thailand,None,totinternet.net,eth:ethertype:ip:tcp:http,eth:ethertype:ip:tcp:http:data-text-lines:xml,[[12:D/EmHXq/Ish+j51h6Y1awh5UP+/4D7eh2xmMReUuC...,"[[3:KD+U/kIgfFf+S5aPS:KKAk7flea, 1578591683.93..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
850403557510,1.578670e+09,"[1578669964.951895, 1578669964.95205, 15786699...",1.085147,6,4,616,904,NaN,NaN,NaN,...,210.242.149.161,48353,80,France,None,ip-94-23-208.eu,eth:ethertype:ip:tcp:http,eth:ethertype:ip:tcp:http:data,[[12:tA3u01vH7BBBBBBBBBBLU1VKZadkWqu8Tg7EkWquH...,[[24:KtoUNnzxT4B1BBBBBBBBBBWaKBNAs+rsqDtdbNes8...
850403557518,1.578609e+09,"[1578609108.047407, 1578609108.047686, 1578609...",3.854807,5,5,570,784,NaN,NaN,NaN,...,210.242.148.70,42278,80,Italy,None,telecomitalia.it,eth:ethertype:ip:tcp:http,eth:ethertype:ip:tcp:http:data-text-lines:xml,[[12:DC779Xdykq16Bn6R7o6sD/XmHXqrI2ErB5cuXNQNc...,"[[3:KD+U/kIgfFf+S5aPS:KKAk7flea, 1578609110.27..."
850403557520,1.578649e+09,"[1578649151.746434, 1578649151.746658, 1578649...",110.289719,11,15,1705,1038,NaN,NaN,NaN,...,210.242.147.61,37037,8080,Kazakhstan,None,None,eth:ethertype:ip:tcp:http,eth:ethertype:ip:tcp:http:data,[[12:rHXq6xmMRe6ShIshSR56Y1awh5UPQVJJtzpvhm:Da...,"[[3:Sn:Sn, 1578649174.522186000, 2], [3:Sn:Sn,..."
850403557552,1.578596e+09,"[1578595622.858003, 1578595622.858309, 1578595...",4.480186,26,19,14660,22232,NaN,NaN,NaN,...,210.242.149.162,37488,80,Russia,None,corbina.ru,eth:ethertype:ip:tcp:http:urlencoded-form,eth:ethertype:ip:tcp:http:data-text-lines,[[24:WAsUiTAawnUHrCWS18YH+w18YHxEMgz+MgtAsUiAl...,[[96:cb4FqCp4QuQxr94OjqozJnsnSnrnb0xWgUQCx3R:q...


In [58]:
all_session_num = []
all_packet_num = []
all_ip_num = []
all_ip_num2 = []
all_country_num = []
all_country_num2 = []
for i,(k,v) in enumerate(proto_big_dict.items()):
    need_keys = v[:]
    need_keys.append(k)
    temp_df = proto_df.loc[need_keys]
    
    session_num = len(temp_df)
#     session_num = len(list(functools.reduce(operator.iconcat, session_li, [])))
    all_session_num.append(session_num)
    packet_num = temp_df['session_i_tt_packet'].sum() + temp_df['session_o_tt_packet'].sum()
    all_packet_num.append(packet_num)
    temp = temp_df['ip_src'].tolist()
    ip_num = len(set(temp))
    all_ip_num2.append(ip_num)
    all_ip_num.extend(list(set(temp)))
    temp = temp_df['country'].tolist()
    country_num = len(set(temp))
    all_country_num2.append(country_num)
    all_country_num.extend(list(set(temp)))
    if i<6:
        print(session_num,packet_num,ip_num,country_num)
for k in proto_loners:
    need_keys.append(k)
    temp_df = proto_df.loc[k]

    session_num = len(temp_df)
#     session_num = len(list(functools.reduce(operator.iconcat, session_li, [])))
    all_session_num.append(session_num)
    packet_num = temp_df['session_i_tt_packet'].sum() + temp_df['session_o_tt_packet'].sum()
    all_packet_num.append(packet_num)
    temp = [temp_df['ip_src']]#.tolist()
    ip_num = len(set(temp))
    all_ip_num2.append(ip_num)
    all_ip_num.extend(list(set(temp)))
    temp = [temp_df['country']]#.tolist()
    country_num = len(set(temp))
    all_country_num2.append(country_num)
    all_country_num.extend(list(set(temp)))    
print('TOTAL session#:',sum(all_session_num),'Total packet#:',sum(all_packet_num),
      'Total of ditinct ips:',len(all_ip_num),'Distict of all ip#:',len(set(all_ip_num)),
     'Total of distict countries:',len(all_country_num),'Distinct if all country#:',len(set(all_country_num))) 
# proto_df.loc[14810]
# temp_df

84863 127987118 1605 3
58578 74124642 3472 97
35648 2427804 5880 128
15238 1070925 2118 101
12270 152263 58 21
9186 2082196 1165 89
TOTAL session#: 238584 Total packet#: 212618040 Total of ditinct ips: 17974 Distict of all ip#: 14263 Total of distict countries: 1303 Distinct if all country#: 154


In [72]:
one = pd.DataFrame.from_dict(dict(Counter(all_session_num)),orient='index')
one = one.reset_index()
one.columns = ['01','02']
two = pd.DataFrame.from_dict(dict(Counter(all_packet_num)),orient='index')
two = two.reset_index()
two.columns = ['03','04']
three = pd.DataFrame.from_dict(dict(Counter(all_ip_num2)),orient='index')
three = three.reset_index()
three.columns = ['05','06']
four = pd.DataFrame.from_dict(dict(Counter(all_country_num2)),orient='index')
four = four.reset_index()
four.columns = ['07','08']
save_df = pd.DataFrame([all_session_num,all_packet_num,all_ip_num2,all_country_num2]).T
save_df.columns = ['session_num','packet_num','ip_num','country_num']
for df in [one,two,three,four]:
    save_df = pd.concat([save_df,df],join='outer')
save_df.to_excel('/home/antslab/NAS1_RAID6/pcap_inter/2020_01_10/中華電信/case_pictures/http_20200110_dist_diagram.xlsx') #改!!
save_df

,session_num,packet_num,ip_num,country_num,01,02,03,04,05,06,07,08
0,84863.0,127987118.0,1605.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,58578.0,74124642.0,3472.0,97.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,35648.0,2427804.0,5880.0,128.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,15238.0,1070925.0,2118.0,101.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,12270.0,152263.0,58.0,21.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,4.0
22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,3.0
23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.0,3.0
24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.0,1.0


In [70]:
one.reset_index()

,index,session_count
0,84863,1
1,58578,1
2,35648,1
3,15238,1
4,12270,1
...,...,...
67,6,9
68,5,8
69,4,10
70,3,14


In [66]:
one = pd.DataFrame.from_dict(dict(Counter(all_session_num)),orient='index')
pd.concat([one,save_df],join='outer')
# save_df.merge(one,how='outer',left_index=False,right_index=False)

,0,country_num,ip_num,packet_num,session_num
84863,1.0,NaN,NaN,NaN,NaN
58578,1.0,NaN,NaN,NaN,NaN
35648,1.0,NaN,NaN,NaN,NaN
15238,1.0,NaN,NaN,NaN,NaN
12270,1.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...
165,NaN,1.0,1.0,64.0,30.0
166,NaN,1.0,1.0,81.0,30.0
167,NaN,1.0,1.0,32.0,30.0
168,NaN,1.0,1.0,43.0,30.0


In [40]:
len(proto_loners) + len(proto_big_dict)

170

156

In [14]:
pd.DataFrame.from_dict(proto_big_dict,orient='index')

,0,1,2,3,4,5,6,7,8,9,...,84852,84853,84854,84855,84856,84857,84858,84859,84860,84861
14810,609885387293,6.528351e+11,6.614250e+11,6.700149e+11,6.786049e+11,6.871948e+11,6.012957e+10,8.589938e+10,9.448931e+10,1.030792e+11,...,6.012955e+11,6.098854e+11,7.473243e+11,6.528351e+11,6.614250e+11,6.786049e+11,6.700149e+11,4.123169e+11,4.294968e+11,5.239860e+11
354,197568495616,1.803886e+11,6.270652e+11,2.662880e+11,5.927055e+11,4.294967e+10,2.405182e+11,2.748779e+11,2.834678e+11,9.448928e+10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
117,824633720833,3.178276e+11,1.975685e+11,1.717987e+10,1.717987e+10,5.927055e+11,8.589935e+09,8.589935e+09,1.632088e+11,1.632088e+11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
243,32768,3.276900e+04,3.277000e+04,3.277100e+04,7.730944e+10,7.730944e+10,1.717990e+10,6.012957e+10,1.717990e+10,1.202591e+11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
242,77309444098,7.730944e+10,1.717990e+10,6.012957e+10,1.717990e+10,6.012957e+10,7.730944e+10,5.669357e+11,5.927055e+11,5.669357e+11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
609885386225,790274013278,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
609885386240,790274013230,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
635655179070,781684069004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
652835048336,695784718881,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
